❗TODO
---

⚙ Настройки
---

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !nvidia-smi

In [ ]:
#@title Настройки картинок { run: "auto", vertical-output: true }

Dim_XY = 101 #@param {type:"integer"} # Размеры картинок 
Padding = 25 #@param {type:"integer"} # Размер паддинга


# pics_folder_Path = "/content/drive/MyDrive/pics/"
pics_folder_Path = "http://s-art.pro/диплом/"
pics_Name = "george" #@param ["lenna", "george", "черепаха"]
# pics_Path = pics_folder_Path+'/'+pics_Name+'/'
pics_Path = pics_folder_Path+'/'+pics_Name+'.png'

#@markdown 
#@markdown Замена картинки на белое изображение
Real_Zeros_ON = False #@param {type:"boolean"}
Imag_Zeros_ON = True #@param {type:"boolean"}

#@markdown 
#@markdown Поменять местами картинки
Real_Imag_Swap_ON = False #@param {type:"boolean"}

In [ ]:
#@title Загрузка/Тренировка/Сохранение модели { run: "auto", vertical-output: true }
Load_ON = False #@param {type:"boolean"}
Train_ON = True #@param {type:"boolean"}
Save_ON = False #@param {type:"boolean"}
postfix = "_test" #@param {type:"string"}

model_folder_Path = '/content/drive/MyDrive/models/' # Путь до сохраненных моделей
model_filename = 'model_Phase Problem ANN Conv v1.2 ' + pics_Name + ' ' + str(Dim_XY)+'x'+str(Dim_XY)+' padding'+str(Padding)+postfix+'.pt'


In [ ]:
#@title Параметры обучения { run: "auto", vertical-output: true }
epochs_Num = 15000//2#@param {type:"integer"} # Количество эпох
loss_break = 0.00 #@param {type:"number"} # Значение loss для принудительной остановки


🧰 Импорты и утилити функции
---

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import pandas as pd

# !pip install torchinfo 
# !pip install torchviz
# from torchinfo import summary
# from torchviz import make_dot


import math
import numpy as np
import itertools

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import random

import requests


MSE = nn.MSELoss(reduction='mean')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print(device)

In [ ]:
import matplotlib.colors as colors
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Функция расчета разницы между фазами на кольце [0, 2pi]
def ring_alg(a,b):
    return math.pi - np.abs(np.abs(a-b)-math.pi)

def plot_image(image, axis, logscale=False, colorbar=True, vmin=None, vmax=None, interpolation='nearest', title = '', fontsize = 20, **kwargs):
    """Plot image to axis with optional logscale and colorbar

    Keyword arguments:
    image -- 2D array with data to show
    axis -- matplotlib axis to plot
    logscale -- use logscale
    colorbar -- add colorbar to image
    vmin -- minimal value of colormap
    vmax -- miximum value of colormap
    """
    if vmin is None and logscale:
        vmin = max(1, np.amin(image))
    elif vmin is None:
        vmin = np.amin(image)

    if vmax is None:
        vmax = np.amax(image)

    if logscale:
        norm = colors.LogNorm(vmin=vmin, vmax=vmax)
    else:
        norm = colors.Normalize(vmin=vmin, vmax=vmax)

    im_handle = axis.imshow(image, norm=norm, interpolation=interpolation, **kwargs)
    axis.set_xticks([])
    axis.set_yticks([])

    axis.set_title(title, fontsize=fontsize)

    if colorbar:
        divider = make_axes_locatable(axis)
        cb_axis = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im_handle, cax=cb_axis)


def plot_image_3D(image, axis, logscale=False, colorbar=True, vmin=None, vmax=None, interpolation='nearest', title = '', fontsize = 20, **kwargs):
    """Plot image to axis with optional logscale and colorbar

    Keyword arguments:
    image -- 2D array with data to show
    axis -- matplotlib axis to plot
    logscale -- use logscale
    colorbar -- add colorbar to image
    vmin -- minimal value of colormap
    vmax -- miximum value of colormap
    """
    if vmin is None and logscale:
        vmin = max(1, np.amin(image))
    elif vmin is None:
        vmin = np.amin(image)

    if vmax is None:
        vmax = np.amax(image)

    if logscale:
        norm = colors.LogNorm(vmin=vmin, vmax=vmax)
    else:
        norm = colors.Normalize(vmin=vmin, vmax=vmax)

    # Make data.
    X, Y = np.arange(0, image.shape[0],1), np.arange(0, image.shape[1],1)
    X, Y = np.meshgrid(X, Y)
    Z = image
    surf = axis.plot_surface(X, Y, Z,linewidth=0, antialiased=False, norm=norm, **kwargs)

    axis.set_xticks([])
    axis.set_yticks([])

    axis.set_title(title, fontsize=fontsize)

    # if colorbar:
    #     divider = make_axes_locatable(axis)
    #     cb_axis = divider.append_axes("bottom", size="5%", pad=0.05)
    #     # plt.colorbar(surf, cax=cb_axis)
    #     plt.colorbar(surf)

In [ ]:
def fftn_shift(real_data, dim=(-2,-1)):
    return torch.fft.fftshift(torch.fft.fftn(torch.fft.ifftshift(real_data, dim=dim), dim=dim), dim=dim)

def ifftn_shift(recip_data, dim=(-2,-1)):
    return torch.fft.fftshift(torch.fft.ifftn(torch.fft.ifftshift(recip_data, dim=dim), dim=dim), dim=dim)

# def fftn_shift(real_data):
#     return torch.fft.fftn(real_data, norm='ortho')

# def ifftn_shift(recip_data):
#     return torch.fft.ifftn(recip_data, norm='ortho')

🖼 Составляем входные данные
---

In [ ]:
# if Real_Imag_Swap_ON:
#   real_Arr = np.asarray(Image.open(pics_Path+'imag.png').resize((Dim_XY, Dim_XY), Image.ANTIALIAS).convert('L'))
#   # imag_Arr = np.asarray(Image.open(pics_Path+'real.png').resize((Dim_XY, Dim_XY), Image.ANTIALIAS).convert('L'))
# else:
#   real_Arr = np.asarray(Image.open(pics_Path+'real.png').resize((Dim_XY, Dim_XY), Image.ANTIALIAS).convert('L'))
#   # imag_Arr = np.asarray(Image.open(pics_Path+'imag.png').resize((Dim_XY, Dim_XY), Image.ANTIALIAS).convert('L'))

real_Arr = np.asarray(Image.open(requests.get(pics_Path, stream=True).raw).resize((Dim_XY, Dim_XY), Image.ANTIALIAS).convert('L'))


# Нормируем картинки
real_Arr = real_Arr/255


# Заменяем картинки на белый фон
if Real_Zeros_ON:
    real_Arr = np.zeros((Dim_XY, Dim_XY))
if Imag_Zeros_ON:
    imag_Arr = np.zeros((Dim_XY, Dim_XY))


# Добавляем Паддинг
real_Arr_no_pad = real_Arr.copy()
imag_Arr_no_pad = imag_Arr.copy()

real_Arr = np.pad(real_Arr, Padding)
imag_Arr = np.pad(imag_Arr, Padding)

# Создаем маску паддинга
mask_Arr = np.pad(np.full_like(real_Arr_no_pad, False), Padding, mode='constant', constant_values=True) 


fig_orig, ax_orig = plt.subplots(1,2,figsize=(14,8))

plot_image(real_Arr, ax_orig[0], cmap='gray')
plot_image(imag_Arr, ax_orig[1], cmap='gray')

---
1. Преобразуем данные в тензоры
2. Вычисляем тензор амплитуд и фаз
3. Вычисляем тензор амплитуд и фаз фурье-образа
4. Аналогично делаем для транспонированных картинок

In [ ]:
real_Ten = torch.from_numpy(real_Arr).to(torch.float32).to(device) # Скрытая от ИНН информация
imag_Ten = torch.from_numpy(imag_Arr).to(torch.float32).to(device) # Скрытая от ИНН информация
mask_Ten = torch.from_numpy(mask_Arr).to(torch.bool).to(device) # Известная ИНН информация 

full_Ten = real_Ten+imag_Ten

ampl_Ten = torch.abs(full_Ten) # Известная ИНН информация 
phase_Ten = torch.angle(full_Ten) # Скрытая от ИНН информация
f_ampl_Ten = torch.abs(fftn_shift(full_Ten)) # Известная ИНН информация 
f_phase_Ten = torch.angle(fftn_shift(full_Ten)) # Скрытая от ИНН информация 
print(f_phase_Ten) 

fig_ampl, ax_ampl = plt.subplots(1, figsize=(14,8))
plot_image(ampl_Ten.detach().cpu().numpy(), ax_ampl, cmap='gray')

fig_f_ampl, ax_f_ampl = plt.subplots(1, figsize=(14,8))
plot_image(f_ampl_Ten.detach().cpu().numpy(), ax_f_ampl, logscale=True)

fig_f_phase, ax_f_phase = plt.subplots(1, figsize=(14,8))
plot_image(f_phase_Ten.detach().cpu().numpy(), ax_f_phase, cmap='twilight')

In [ ]:
real_Ten_no_pad = torch.from_numpy(real_Arr_no_pad).to(torch.float32).to(device) # Скрытая от ИНН информация
imag_Ten_no_pad = torch.from_numpy(imag_Arr_no_pad).to(torch.float32).to(device)*1j # Скрытая от ИНН информация

full_Ten_no_pad = real_Ten_no_pad+imag_Ten_no_pad

ampl_Ten_no_pad = torch.abs(full_Ten_no_pad) # Известная ИНН информация 
phase_Ten_no_pad = torch.angle(full_Ten_no_pad) # Скрытая от ИНН информация
f_ampl_Ten_no_pad = torch.abs(fftn_shift(full_Ten_no_pad)) # Известная ИНН информация 
f_phase_Ten_no_pad = torch.angle(fftn_shift(full_Ten_no_pad)) # Скрытая от ИНН информация 

fig_ampl, ax_ampl = plt.subplots(1, figsize=(14,8))
plot_image(ampl_Ten_no_pad.detach().cpu().numpy(), ax_ampl, cmap='gray')

fig_f_ampl, ax_f_ampl = plt.subplots(1, figsize=(14,8))
plot_image(f_ampl_Ten_no_pad.detach().cpu().numpy(), ax_f_ampl, logscale=True)

fig_f_phase, ax_f_phase = plt.subplots(1, figsize=(14,8))
plot_image(f_phase_Ten_no_pad.detach().cpu().numpy(), ax_f_phase, cmap='twilight')

In [ ]:
# Смотрим математическое ожидание среднего I, сгенерированного через |I'| со случайными фазами (Дебаг информация)
fig_ampl, ax_ampl = plt.subplots(1, figsize=(14,8))
plot_image(ifftn_shift(torch.polar(f_ampl_Ten_no_pad, torch.rand_like(f_ampl_Ten_no_pad)*2*math.pi)).real.detach().cpu().numpy(), ax_ampl, cmap='gray', vmin=-1,vmax=1)
print(ifftn_shift(torch.polar(f_ampl_Ten_no_pad, torch.rand_like(f_ampl_Ten_no_pad)*2*math.pi)).real.detach().cpu().numpy().mean())

🤖 Создаем версию нейронной сети
---


In [ ]:
Dim_XY_P = Dim_XY+Padding*2

# class PhaseSolver(torch.nn.Module):
#     def __init__(self):
#       super().__init__()
#       lin_list = [] # Лист для линейных слоев вида 1->K(i) 
#       if Dim_XY_P % 2 == 0: # Отдельно расчитываем длины квадратов для четных и нечетных размеров NxN
#         for i in range(Dim_XY_P//2):
#           self.temp_lin = torch.nn.Linear(1, (i*2+1)*4) # Создаем слои по формуле периметра квадрата заданного номера
#           lin_list.append(self.temp_lin)
#       else:
#         lin_list.append(torch.nn.Linear(1, 1)) # Отдельно создаем слой центрального пикселя фаз 1->1
#         for i in range(1, Dim_XY_P//2+1):
#           self.temp_lin = torch.nn.Linear(1, (i*2)*4) # Создаем слои по формуле периметра квадрата заданного номера
#           lin_list.append(self.temp_lin)
#       self.lin_list = nn.ModuleList(lin_list) # Добавляем параметры слоев в параметры класса нейросети. 
      
#       # Расчитываем массив перестановки, который преобразует массив [Пиксели центрального слоя фаз, Пиксели первого слоя фаз ... Пиксели последнего слоя фаз]
#       # в массив развернутого изображения. То есть такой массив, что массив.view(Dim_XY_P,Dim_XY_P) будет является правильным изображением фаз
#       # Для этого создаем матрицу формата
#       # 2 2 2 2 2 2
#       # 2 1 1 1 1 2
#       # 2 1 0 0 1 2
#       # 2 1 0 0 1 2
#       # 2 1 1 1 1 2
#       # 2 2 2 2 2 2
#       # Которую разворачиваем в один слой, а затем ищем перестановку, превращющую его в 0,0,0,0,1,..,1,2,..,2. Argsort делает именно это.
#       # Для этой перестановки ищем обратную перестановку, используя свойство перестановка.argsort() = обратная_перестановка. 
#       if Dim_XY_P % 2 == 0: 
#         perm_T = torch.zeros((2,2), requires_grad=False)
#         for i in range(1, Dim_XY_P//2):
#           perm_T = torch.nn.functional.pad(perm_T, (1,1,1,1), value=i) # Составляем четный вариант массива для нахождения перестановки
#       else:
#         perm_T = torch.zeros((1,1), requires_grad=False)
#         for i in range(1, Dim_XY_P//2 +1 ):
#           perm_T = torch.nn.functional.pad(perm_T, (1,1,1,1), value=i) # Составляем нечетный вариант массива для нахождения перестановки
#       self.perm_T = perm_T
#       self.perm = torch.argsort(torch.argsort(perm_T.view(-1))) # Ищем обратную перестановку
    
#     def forward(self, x, n_sqr):
#       out = [] # Лист езультатов линейных слоев.
#       out.append(self.lin_list[0](x)) # Безусловно считаем центральные пиксели.
#       for i in range(n_sqr-1): # Считаем указанное в форвард количество квадратов.
#         out.append(self.lin_list[i](x))
#       out = torch.cat(out, dim=0) # Складываем их в один тензор 
      
#       # Дополняем тензор нулями до размеров изображения, делаем перестановку, расчитанную на инициализации, решейпим массив в матрицу.
#       out = torch.nn.functional.pad(out, (0,Dim_XY_P**2 - out.size(0)))[self.perm].view(Dim_XY_P, Dim_XY_P)

#       out = out*math.pi

#       # Делаем матрицу антисимметричной 
#       out = self.antisimm(out)

#       # Выполняем цикл HIO
#       # out = self.HIO(out, hio_N=2)

#       return out

#     # Делает двумерную квадратную матрицу антисимметричной
#     def antisimm(self, x):
#       diag = x.diag().diag().flip([-1]).triu(0)
#       diag = diag.flip([-1]) - diag.flip([-2])
#       x = torch.triu(x,diagonal=1) - torch.triu(x,diagonal=1).flip([-1,-2]) + diag
#       return x

#     # На вход нужно подать приближение фаз обратного пространства и правильные амплитуды обратного пространства. На выходе получаем приближение фаз.
#     def HIO(self, f_phase_x, f_ampl_x = f_ampl_Ten, hio_N = 0, era_N = 0, beta=0.9, give_kristall_fl = False):
#         if hio_N == 0 and era_N == 0:
#             return f_phase_x
        
#         iter_N = hio_N + era_N
        
#         x = ifftn_shift(torch.polar(f_ampl_x, f_phase_x))
#         f_ampl_x = f_ampl_x.abs()

#         # Расчитываем маску паддинга
#         pad_mask = torch.full_like(x.abs(), True).to(torch.bool)
#         pad_size = int(x.size(-1)/Dim_XY_P*Padding)
#         pad_mask[pad_size:-pad_size,pad_size:-pad_size] = torch.full_like(x.abs(), False).to(torch.bool)[pad_size:-pad_size,pad_size:-pad_size]

#         x_past = x.real
#         x[pad_mask] = 0

#         for i in range(iter_N):
#             hio = (i % (iter_N)) < hio_N
#             # Подставялем известные апмлитуды и забираем приближенное изображение
#             x = ifftn_shift(torch.polar(f_ampl_x, f_phase_x))

#             # Расчитываем маску неположительных значений
#             iter_mask = torch.full_like(x.real, False).to(torch.bool)
#             iter_mask[x.real<=0] = True
#             iter_mask = torch.logical_or(pad_mask, iter_mask)

#             # Приближаем к 0 по маске
#             if hio:
#                   x[iter_mask] = x[iter_mask] - x_past[iter_mask]*beta
#             else:
#                   x[iter_mask] = 0
            
#             # Запоминаем предыдущий результат 
#             x_past = x

#             # Вычисляем новое приблжение фаз
#             f_phase_x = torch.angle(fftn_shift(x))
        
#         if give_kristall_fl:
#             return ifftn_shift(torch.polar(f_ampl_x, f_phase_x))
#         return f_phase_x
# PS = PhaseSolver().to(device)


# На вход нужно подать приближение фаз обратного пространства и правильные амплитуды обратного пространства. На выходе получаем приближение фаз.
def HIO_func(f_phase_x, f_ampl_x = f_ampl_Ten, hio_N = 0, era_N = 0, beta=0.9, give_kristall_fl = False):
    if hio_N == 0 and era_N == 0:
        return f_phase_x
    
    iter_N = hio_N + era_N
    
    x = ifftn_shift(torch.polar(f_ampl_x, f_phase_x))
    f_ampl_x = f_ampl_x.abs()

    # Расчитываем маску паддинга
    pad_mask = torch.full_like(x.abs(), True).to(torch.bool)
    pad_size = int(x.size(-1)/Dim_XY_P*Padding)
    pad_mask[pad_size:-pad_size,pad_size:-pad_size] = torch.full_like(x.abs(), False).to(torch.bool)[pad_size:-pad_size,pad_size:-pad_size]

    x_past = x.real
    x[pad_mask] = 0

    for i in range(iter_N):
        hio = (i % (iter_N)) < hio_N
        # Подставялем известные апмлитуды и забираем приближенное изображение
        x = ifftn_shift(torch.polar(f_ampl_x, f_phase_x))

        # Расчитываем маску неположительных значений
        iter_mask = torch.full_like(x.real, False).to(torch.bool)
        iter_mask[x.real<=0] = True
        iter_mask = torch.logical_or(pad_mask, iter_mask)

        # Приближаем к 0 по маске
        if hio:
              x[iter_mask] = x[iter_mask] - x_past[iter_mask]*beta
        else:
              x[iter_mask] = 0
        
        # Запоминаем предыдущий результат 
        x_past = x

        # Вычисляем новое приблжение фаз
        f_phase_x = torch.angle(fftn_shift(x))
    
    if give_kristall_fl:
        return ifftn_shift(torch.polar(f_ampl_x, f_phase_x))
    return f_phase_x

# ANN_train_res_PhaseSolver = (torch.rand_like(f_phase_Ten).to(device) * 2 - 1)*math.pi

ANN_train_res_PhaseSolver = f_phase_Ten.clone().detach();
print('MSE:', MSE(ANN_train_res_PhaseSolver, f_phase_Ten).item())
while MSE(ANN_train_res_PhaseSolver, f_phase_Ten).item() < 2*math.pi:
    ANN_train_res_PhaseSolver = ANN_train_res_PhaseSolver + ANN_train_res_PhaseSolver*((torch.rand_like(f_phase_Ten).to(device)-0.5)/100 * 5)
    ANN_train_res_PhaseSolver = ANN_train_res_PhaseSolver
ANN_train_res_PhaseSolver = ANN_train_res_PhaseSolver%(2*math.pi)
print('MSE:', MSE(ANN_train_res_PhaseSolver, f_phase_Ten).item())
ANN_train_res_PhaseSolver_init_Arr = ANN_train_res_PhaseSolver.clone().detach().cpu().numpy()
ANN_train_res_PhaseSolver.requires_grad_()

ANN_fig_real, ANN_ax_real = plt.subplots(2,1,figsize=(14,6))
plot_image(ANN_train_res_PhaseSolver_init_Arr, ANN_ax_real[0], cmap='twilight')
plot_image(ifftn_shift(torch.polar(f_ampl_Ten,ANN_train_res_PhaseSolver)).real.detach().cpu().numpy(), ANN_ax_real[1], cmap='gray')
plt.show()

def PS(*args):
    global ANN_train_res_PhaseSolver
    return ANN_train_res_PhaseSolver

n_sqr_Max = Dim_XY_P//2+1+Dim_XY_P%2
n_sqr = n_sqr_Max
Train_Sum_Epochs = 0
Train_Sum_Time = 0

Загружаем модель

In [ ]:
# Load_ON = True
# Load_ON = False
if Load_ON:
    PS = torch.load(model_folder_Path+model_filename)

Инициализируем модель

In [ ]:
one_T = torch.tensor([1]).to(torch.float32).to(device)
test_res = PS(one_T, n_sqr_Max)
# test_res = PS(one_T, 4)

print(test_res.size())
ANN_fig_real, ANN_ax_real = plt.subplots(1,1,figsize=(28,6))
plot_image(test_res.detach().cpu().numpy(), ANN_ax_real, cmap='twilight')
plt.show()
print()
# make_dot(test_res, params=dict(list(PS.named_parameters()))).render("/content/rnn_torchviz", format="png")

Проверяем размеры модели

In [ ]:
# !nvidia-smi
# summary(PS, one_T.size())

### Объявляем массив с информацией для анализа PCA

In [ ]:
PCA_ANN = []
PCA_ANN_LOSS = []
PCA_ANN_HIO = []
PCA_ANN_HIO_LOSS = []

🏃Составляем оптимизатор и функцию ошибки. Запускаем цикл обучения ANN без HIO.
---


In [ ]:
import datetime
last_loss = float('inf')
last_lr = 0.0001

opt_fn = torch.optim.AdamW
# opt_fn = torch.optim.SGD
optimizer = opt_fn([ANN_train_res_PhaseSolver], lr=last_lr)

# optimizer = torch.optim.AdamW([ANN_train_res_PhaseSolver], lr=last_lr)
# optimizer = torch.optim.AdamW(PS.parameters(), lr=last_lr)
# optimizer = torch.optim.Adam(PS.parameters(), lr=0.00001)
# optimizer = torch.optim.NAdam(PS.parameters(), lr=0.0002)
# optimizer = torch.optim.SGD(PS.parameters(), lr=last_lr, momentum = 0.2)
# optimizer = torch.optim.SGD(PS.parameters(), lr=last_lr)

one_T = torch.tensor([1]).to(torch.float32).to(device)
def PS_Loss(f_phas_Ten_Sum, f_ampl_Ten = f_ampl_Ten, f_ampl_Ten_no_pad = f_ampl_Ten_no_pad, mask_Ten = mask_Ten, cheat_ampl_Ten = ampl_Ten):
    ANN_f_result_Ten = torch.polar(f_ampl_Ten, f_phas_Ten_Sum)
    
    ANN_comp_Ten = ifftn_shift(ANN_f_result_Ten)
    ANN_comp_Ten_no_pad = ANN_comp_Ten[Padding:-Padding, Padding:-Padding]
    # ANN_f_ampl_Ten_no_pad = torch.abs(fftn_shift(ANN_comp_Ten_no_pad))

    # loss_phase = MSE(ANN_f_ampl_Ten_no_pad, f_ampl_Ten_no_pad)
    loss_d_phase = MSE(torch.angle(ANN_comp_Ten_no_pad), torch.zeros_like(ANN_comp_Ten_no_pad.real))
    loss_nega = MSE(ANN_comp_Ten_no_pad.real[ANN_comp_Ten_no_pad.real<0], torch.zeros_like(ANN_comp_Ten_no_pad.real[ANN_comp_Ten_no_pad.real<0]))
    loss_nega = torch.nan_to_num(loss_nega)
    loss_imag = MSE(ANN_comp_Ten_no_pad.imag, torch.zeros_like(ANN_comp_Ten_no_pad.imag))
    loss_pad = MSE(ANN_comp_Ten.real[mask_Ten], torch.zeros_like(ANN_comp_Ten.real[mask_Ten])) *1000
    loss_cheat = MSE(ANN_comp_Ten.real*255,cheat_ampl_Ten*255)

    

    # Проверяем, что переворот изображения максимально отличается от самого изображения =========================================================== (*)
    # После ряда экспериментов, видим, что этот лосс дает около 8(+/-3) на инициализации
    loss_mirror = (1/MSE(ANN_comp_Ten_no_pad.real,torch.flip(ANN_comp_Ten_no_pad.real, [0, 1]))) * 0.1 # Корректирующий коэффицент
    


    # loss = loss_phase + loss_imag + loss_pad
    # return loss, loss_imag, loss_phase, loss_pad
    
    # loss = loss_phase + loss_imag
    # return loss, loss_imag, loss_phase

    # loss = loss_imag + loss_pad
    # return loss, loss_imag, loss_pad

    # loss = loss_cheat + loss_pad + loss_d_phase
    # loss = loss_pad + loss_d_phase + loss_mirror
    # loss = loss_pad + loss_d_phase
    loss = loss_pad + loss_nega + loss_imag + loss_d_phase
    return loss, loss_cheat, loss_pad, loss_d_phase, loss_mirror, loss_nega, loss_imag

def train_model(epochs_Num):
  global Train_Sum_Epochs
  global Train_Sum_Time
  global optimizer
  global last_loss
  global last_lr
  global n_sqr
  global ANN_train_res_PhaseSolver

  ANN_f_phas_Arr_Last = PS(one_T, 1).detach().cpu().numpy()
  hio_flag = True
  
  for t in range(epochs_Num+1):  
      Epoch_Time = datetime.datetime.now()

      optimizer.zero_grad()

      optimizer.zero_grad()
      NN_out = PS(one_T, n_sqr)

      if MSE(ifftn_shift(torch.polar(f_ampl_Ten, NN_out)).real, ampl_Ten) > MSE(ifftn_shift(torch.polar(f_ampl_Ten, -NN_out)).real, ampl_Ten):
        NN_out = -NN_out

      # loss, loss_phase, loss_imag, loss_pad = PS_Loss(f_phas_Ten)
      # loss, loss_phase, loss_imag = PS_Loss(f_phas_Ten)
      # loss, loss_imag, loss_pad = PS_Loss(f_phas_Ten)
      loss, loss_cheat, loss_pad, loss_d_phas, loss_mirror, loss_nega, loss_imag = PS_Loss(NN_out)
      # _, loss_cheat, _, _, _ = PS_Loss(NN_out[10:-10,10:-10], )
      # loss = PS_Loss(f_phas_Ten)
      
      # PCA
      if n_sqr >= n_sqr_Max:
        PCA_ANN.append(NN_out.clone().detach().cpu().numpy())
        PCA_ANN_LOSS.append(loss_cheat.item())

      loss.backward()
      optimizer.step()

      delta_Time = datetime.datetime.now() - Epoch_Time
      Train_Sum_Time += (datetime.datetime.now() - Epoch_Time).total_seconds()
      Train_Sum_Epochs += 1

      if t % (epochs_Num/1000) == 0:
          if Save_ON:
            if t % (epochs_Num/50) == 0:
                torch.save(PS, model_folder_Path+model_filename)
          print('%-4s' % (str(int(t/epochs_Num*1000))+'‰'),
                '- Ep: '+ '%-5s'%str(Train_Sum_Epochs//10*10), 
                '- LSum: '+'%10s'%('%.6f'%loss.item()), 
                # '- LPha: '+'%10s'%('%.6f'%loss_phase.item()), 
                # '- LIma: '+'%10s'%('%.6f'%loss_imag.item()), 
                '- LD_CHEAT: '+'%10s'%('%.6f'%loss_cheat.item()), 
                # '- LDPhas: '+'%10s'%('%.6f'%loss_d_phas.item()), 
                '- LNega: '+'%10s'%('%.6f'%loss_nega.item()), 
                '- LImag: '+'%10s'%('%.6f'%loss_imag.item()), 
                '- LPad: '+'%10s'%('%.6f'%loss_pad.item()), 
                # '- LMir: '+'%10s'%('%.6f'%loss_mirror.item()), 
                '- Sec: '+'%4s'%'%.2f'%round(Train_Sum_Time,2), 
                '- Log10(Lr): ' + '%2s' % str(int(-math.log10(last_lr))), 
                '- Mem: ' + str(torch.cuda.memory_allocated('cuda')))
          

          if t % (epochs_Num/1000*10) == 0:
              ANN_f_phas_Arr = NN_out.clone().detach().cpu().numpy()
              ANN_real_Arr = ifftn_shift(torch.polar(f_ampl_Ten, NN_out)).real.detach().cpu().numpy()
              ANN_f_phas_Arr_Delta = ANN_f_phas_Arr-ANN_f_phas_Arr_Last
              ANN_f_phas_Arr_Delta_20 = ANN_f_phas_Arr_Delta.copy()
              ANN_f_phas_Arr_Delta_20[abs(ANN_f_phas_Arr_Delta)<abs(ANN_f_phas_Arr_Delta).max()*0.8] = 0

              # Выводим фазы фурье-образа картинок
              ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,4, figsize=(28,6), subplot_kw={"projection": "3d"})
              ANN_fig_f_phase.suptitle('Fourie Transform Phases; Квадрат: '+str(n_sqr*2)+'x'+str(n_sqr*2), fontsize=24)
              plot_image_3D(f_phase_Ten.detach().cpu().numpy(), ANN_ax_f_phase[0], cmap='twilight', title="Оригинал")
              plot_image_3D(ANN_f_phas_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Приближение(k)")
              plot_image_3D(ANN_f_phas_Arr_Delta, ANN_ax_f_phase[2], cmap='twilight', title="Разница между k и k-10", alpha=0.5)
              plot_image_3D(ANN_f_phas_Arr_Delta_20, ANN_ax_f_phase[3], cmap='twilight', title="Разница между k и k-10 (20% от max)", alpha=0.5)
              plt.show()
              print()
              # Выводим фазы фурье-образа картинок
              if n_sqr < Dim_XY_P//2+1:
                print_ANN_f_phase_norm = torch.angle(torch.polar(f_ampl_Ten,NN_out))[n_sqr*2:-n_sqr*2,n_sqr*2:-n_sqr*2]
                _pad = (Dim_XY_P-print_ANN_f_phase_norm.size(0))//2
                print_ANN_f_phase_norm = torch.nn.functional.pad(print_ANN_f_phase_norm, (_pad,_pad,_pad,_pad)).detach().cpu().numpy()
              else:
                print_ANN_f_phase_norm = torch.angle(torch.polar(f_ampl_Ten,NN_out)).detach().cpu().numpy()
              ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,5, figsize=(28,6))
              ANN_fig_f_phase.suptitle('Fourie Transform Phases; Квадрат: '+str(n_sqr*2)+'x'+str(n_sqr*2), fontsize=24)
              plot_image(ANN_real_Arr, ANN_ax_f_phase[0], cmap='gray', title="Приближение I")
              plot_image(f_phase_Ten.detach().cpu().numpy(), ANN_ax_f_phase[1], cmap='twilight', title="Оригинал")
              plot_image(print_ANN_f_phase_norm, ANN_ax_f_phase[2], cmap='twilight', title="Приближение I (-Pi, Pi)")
              plot_image(ANN_f_phas_Arr, ANN_ax_f_phase[3], cmap='twilight', title="Приближение(k)")
              plot_image(ANN_f_phas_Arr_Delta, ANN_ax_f_phase[4], cmap='twilight', title="Разница между k и k-10")
              # plot_image(ANN_f_phas_Arr_Delta_20, ANN_ax_f_phase[4], cmap='twilight', title="Разница между k и k-10 (20% от max)")
              plt.show()
              print()
              
              ANN_f_phas_Arr_Last = ANN_f_phas_Arr

          if loss.item() <= loss_break:
              break

          if last_loss < loss.item():
            optimizer.zero_grad()
            last_lr = last_lr/10
            optimizer = opt_fn([ANN_train_res_PhaseSolver], lr=last_lr)
            optimizer.zero_grad()
            last_loss = float('inf')

          last_loss = loss.item()

In [ ]:
if Train_ON:
    Data_Str = '\nКартинки: ' + str(pics_Name) + '\nРазмер: ' + str(Dim_XY)+'x'+str(Dim_XY) + '\nПаддинг: ' + str(Padding)
    print(Data_Str)
    # !nvidia-smi
    train_model(epochs_Num//2)
    # train_model(10)

In [ ]:
phase_n = 150
PCA_ANN_ = np.stack(PCA_ANN[::len(PCA_ANN)//phase_n][-phase_n:])
PCA_ANN_LOSS_ = np.stack(PCA_ANN_LOSS[::len(PCA_ANN_LOSS)//phase_n][-phase_n:])
print(PCA_ANN_.shape)

In [ ]:
pca_ann = PCA(n_components=2)
PCA_ANN_ANS = pca_ann.fit_transform(PCA_ANN_.reshape(PCA_ANN_.shape[0], -1))
print(PCA_ANN_ANS.shape)

In [ ]:
ANN_c = np.stack([[1.0,0,0]]*PCA_ANN_ANS.shape[0])
ANN_c[:,0] = np.log(1 + ANN_c[:,0]*(1-(PCA_ANN_LOSS_-PCA_ANN_LOSS_.min())/(PCA_ANN_LOSS_.max()-PCA_ANN_LOSS_.min())))
plt.scatter(PCA_ANN_ANS[:, 0], PCA_ANN_ANS[:, 1], c=ANN_c)
plt.gca().set(
    title="ANN",
    xlabel="first feature",
    ylabel="second feature",
)
plt.show()


🏃 Запускаем цикл обучения ANN+HIO.
---


In [ ]:
PCA_ANN_HIO = []
PCA_ANN_HIO_LOSS = []

ANN_train_res_PhaseSolver = torch.from_numpy(ANN_train_res_PhaseSolver_init_Arr).clone().to(device)
# ANN_train_res_PhaseSolver = HIO_func(ANN_train_res_PhaseSolver, hio_N=10000, era_N=10000)
ANN_train_res_PhaseSolver.requires_grad_()



In [ ]:
import datetime
last_loss = float('inf')
last_lr = 0.0001

optimizer = opt_fn([ANN_train_res_PhaseSolver], lr=last_lr)

one_T = torch.tensor([1]).to(torch.float32).to(device)
MSE = nn.MSELoss(reduction='mean')

def train_model(epochs_Num):
  global Train_Sum_Epochs
  global Train_Sum_Time
  global optimizer
  global last_loss
  global last_lr
  global n_sqr
  global ANN_train_res_PhaseSolver

  ANN_f_phas_Arr_Last = PS(one_T, 1).detach().cpu().numpy()
  hio_flag = False
  
  for t in range(epochs_Num+1):  
      Epoch_Time = datetime.datetime.now()

      optimizer.zero_grad()
      if (t+1)%150 == 0:
        with torch.no_grad():
          ANN_train_res_PhaseSolver = HIO_func(ANN_train_res_PhaseSolver, hio_N=5, era_N=5)
        ANN_train_res_PhaseSolver.requires_grad_(True)

      optimizer.zero_grad()
      NN_out = PS(one_T, n_sqr)

      if MSE(ifftn_shift(torch.polar(f_ampl_Ten, NN_out)).real, ampl_Ten) > MSE(ifftn_shift(torch.polar(f_ampl_Ten, -NN_out)).real, ampl_Ten):
        NN_out = -NN_out

      # loss, loss_phase, loss_imag, loss_pad = PS_Loss(f_phas_Ten)
      # loss, loss_phase, loss_imag = PS_Loss(f_phas_Ten)
      # loss, loss_imag, loss_pad = PS_Loss(f_phas_Ten)
      loss, loss_cheat, loss_pad, loss_d_phas, loss_mirror, loss_nega, loss_imag = PS_Loss(NN_out)
      # _, loss_cheat, _, _, _ = PS_Loss(NN_out[10:-10,10:-10], )
      # loss = PS_Loss(f_phas_Ten)
      
      # PCA
      if n_sqr >= n_sqr_Max:
        PCA_ANN_HIO.append(NN_out.clone().detach().cpu().numpy())
        PCA_ANN_HIO_LOSS.append(loss_cheat.item())

      loss.backward()
      optimizer.step()

      delta_Time = datetime.datetime.now() - Epoch_Time
      Train_Sum_Time += (datetime.datetime.now() - Epoch_Time).total_seconds()
      Train_Sum_Epochs += 1  

      # if (abs(last_loss-loss.item()) < 0.001):
      #   if n_sqr < n_sqr_Max:
      #     n_sqr += 1
      #     last_lr = 0.0001
      #     optimizer.zero_grad()
      #     optimizer = opt_fn([ANN_train_res_PhaseSolver], lr=last_lr)
      #     optimizer.zero_grad()

      if t % (epochs_Num/1000) == 0:
          if Save_ON:
            if t % (epochs_Num/50) == 0:
                torch.save(PS, model_folder_Path+model_filename)
          print('%-4s' % (str(int(t/epochs_Num*1000))+'‰'),
                '- Ep: '+ '%-5s'%str(Train_Sum_Epochs//10*10), 
                '- LSum: '+'%10s'%('%.6f'%loss.item()), 
                # '- LPha: '+'%10s'%('%.6f'%loss_phase.item()), 
                # '- LIma: '+'%10s'%('%.6f'%loss_imag.item()), 
                '- LD_CHEAT: '+'%10s'%('%.6f'%loss_cheat.item()), 
                # '- LDPhas: '+'%10s'%('%.6f'%loss_d_phas.item()), 
                '- LNega: '+'%10s'%('%.6f'%loss_nega.item()), 
                '- LImag: '+'%10s'%('%.6f'%loss_imag.item()), 
                '- LPad: '+'%10s'%('%.6f'%loss_pad.item()), 
                # '- LMir: '+'%10s'%('%.6f'%loss_mirror.item()), 
                '- Sec: '+'%4s'%'%.2f'%round(Train_Sum_Time,2), 
                '- Log10(Lr): ' + '%2s' % str(int(-math.log10(last_lr))), 
                '- Mem: ' + str(torch.cuda.memory_allocated('cuda')))
          

          if t % (epochs_Num/1000*10) == 0:
              ANN_f_phas_Arr = NN_out.clone().detach().cpu().numpy()
              ANN_real_Arr = ifftn_shift(torch.polar(f_ampl_Ten, NN_out)).real.detach().cpu().numpy()
              ANN_f_phas_Arr_Delta = ANN_f_phas_Arr-ANN_f_phas_Arr_Last
              ANN_f_phas_Arr_Delta_20 = ANN_f_phas_Arr_Delta.copy()
              ANN_f_phas_Arr_Delta_20[abs(ANN_f_phas_Arr_Delta)<abs(ANN_f_phas_Arr_Delta).max()*0.8] = 0

              # Выводим фазы фурье-образа картинок
              ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,4, figsize=(28,6), subplot_kw={"projection": "3d"})
              ANN_fig_f_phase.suptitle('Fourie Transform Phases; Квадрат: '+str(n_sqr*2)+'x'+str(n_sqr*2), fontsize=24)
              plot_image_3D(f_phase_Ten.detach().cpu().numpy(), ANN_ax_f_phase[0], cmap='twilight', title="Оригинал")
              plot_image_3D(ANN_f_phas_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Приближение(k)")
              plot_image_3D(ANN_f_phas_Arr_Delta, ANN_ax_f_phase[2], cmap='twilight', title="Разница между k и k-10", alpha=0.5)
              plot_image_3D(ANN_f_phas_Arr_Delta_20, ANN_ax_f_phase[3], cmap='twilight', title="Разница между k и k-10 (20% от max)", alpha=0.5)
              plt.show()
              print()
              # Выводим фазы фурье-образа картинок
              if n_sqr < Dim_XY_P//2+1:
                print_ANN_f_phase_norm = torch.angle(torch.polar(f_ampl_Ten,NN_out))[n_sqr*2:-n_sqr*2,n_sqr*2:-n_sqr*2]
                _pad = (Dim_XY_P-print_ANN_f_phase_norm.size(0))//2
                print_ANN_f_phase_norm = torch.nn.functional.pad(print_ANN_f_phase_norm, (_pad,_pad,_pad,_pad)).detach().cpu().numpy()
              else:
                print_ANN_f_phase_norm = torch.angle(torch.polar(f_ampl_Ten,NN_out)).detach().cpu().numpy()
              ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,5, figsize=(28,6))
              ANN_fig_f_phase.suptitle('Fourie Transform Phases; Квадрат: '+str(n_sqr*2)+'x'+str(n_sqr*2), fontsize=24)
              plot_image(ANN_real_Arr, ANN_ax_f_phase[0], cmap='gray', title="Приближение I")
              plot_image(f_phase_Ten.detach().cpu().numpy(), ANN_ax_f_phase[1], cmap='twilight', title="Оригинал")
              plot_image(print_ANN_f_phase_norm, ANN_ax_f_phase[2], cmap='twilight', title="Приближение I (-Pi, Pi)")
              plot_image(ANN_f_phas_Arr, ANN_ax_f_phase[3], cmap='twilight', title="Приближение(k)")
              plot_image(ANN_f_phas_Arr_Delta, ANN_ax_f_phase[4], cmap='twilight', title="Разница между k и k-10")
              # plot_image(ANN_f_phas_Arr_Delta_20, ANN_ax_f_phase[4], cmap='twilight', title="Разница между k и k-10 (20% от max)")
              plt.show()
              print()
              
              ANN_f_phas_Arr_Last = ANN_f_phas_Arr

          if loss.item() <= loss_break:
              break
          
          if last_loss < loss.item():
            if hio_flag:
              optimizer.zero_grad()
              with torch.no_grad():
                ANN_train_res_PhaseSolver = HIO_func(ANN_train_res_PhaseSolver, hio_N=50, era_N=50)
              ANN_train_res_PhaseSolver.requires_grad_(True)
              optimizer.zero_grad()
            else:
              optimizer.zero_grad()
              last_lr = last_lr/10
              optimizer = opt_fn([ANN_train_res_PhaseSolver], lr=last_lr)
              optimizer.zero_grad()
            last_loss = float('inf')
            hio_flag = not hio_flag

          last_loss = loss.item()

In [ ]:
if Train_ON:
    Data_Str = '\nКартинки: ' + str(pics_Name) + '\nРазмер: ' + str(Dim_XY)+'x'+str(Dim_XY) + '\nПаддинг: ' + str(Padding)
    print(Data_Str)
    # !nvidia-smi
    train_model(epochs_Num//2)
    # train_model(10)

In [ ]:
phase_n = 150
PCA_ANN_HIO_ = np.stack(PCA_ANN_HIO[::len(PCA_ANN_HIO)//phase_n][-phase_n:])
PCA_ANN_HIO_LOSS_ = np.stack(PCA_ANN_HIO_LOSS[::len(PCA_ANN_HIO_LOSS)//phase_n][-phase_n:])
print(PCA_ANN_HIO_.shape)

In [ ]:
pca_ann = PCA(n_components=2)
PCA_ANN_HIO_ANS = pca_ann.fit_transform(PCA_ANN_HIO_.reshape(PCA_ANN_HIO_.shape[0], -1))
print(PCA_ANN_HIO_ANS.shape)

In [ ]:
ANN_HIO_c = np.stack([[0.0,1.0,0]]*PCA_ANN_HIO_ANS.shape[0])
ANN_HIO_c[:,1] = np.log(1 + ANN_HIO_c[:,1]*(1-(PCA_ANN_HIO_LOSS_-PCA_ANN_HIO_LOSS_.min())/(PCA_ANN_HIO_LOSS_.max()-PCA_ANN_HIO_LOSS_.min())))
plt.scatter(PCA_ANN_HIO_ANS[:, 0], PCA_ANN_HIO_ANS[:, 1], c=ANN_HIO_c)
plt.gca().set(
    title="ANN_HIO",
    xlabel="first feature",
    ylabel="second feature",
)
plt.show()

🎁Сохраняем модель
---

In [ ]:
if Save_ON:
    torch.save(PS, model_folder_Path+model_filename)

⚛ Рисуем получившиеся картинки
---

Угаданные действ. и мним. части изображения перед выводом отдельно abs().

---
1. Сдвиг фазы обязательно нужно подобрать, иначе неверно считаются дельты.
2. Порядок вывода: Угаданное, Оригинал, Дельта
3. Нажатие на картинку ее увеличивает, но проще открыть в отдельной вкладке.

In [ ]:
#@title Настройки вывода
    
def init_graphs(phase_shift):
    global ANN_real_Arr
    global ANN_imag_Arr
    global ANN_phase_Arr
    global ANN_f_ampl_Arr
    global ANN_f_phase_Arr
    global ANN_nopad_f_phase_Arr
    global nopad_real_Arr
    global nopad_imag_Arr
    global nopad_phase_Arr
    global nopad_f_phase_Arr
    global f_phase_Arr
    
    # Сдвиг фазы для верного сопостовления действительной и мнимой частей
    #@markdown Сдвиг фазы происходит на pi/2 * phase_shift
    # Подготавливаем данные
    ANN_phase_res = PS(one_T, Dim_XY_P//2+1)+math.pi/2*phase_shift
    ANN_f_phase_Arr = ANN_phase_res.detach().cpu().numpy()
    ANN_f_result_Ten = torch.polar(f_ampl_Ten,ANN_phase_res)
    ANN_result_Ten = ifftn_shift(ANN_f_result_Ten)
    ANN_f_ampl_Arr = fftn_shift(ANN_result_Ten).abs().detach().cpu().numpy()
    if Padding > 0: # Убираем паддинг
        ANN_result_Ten = ANN_result_Ten[Padding:-Padding, Padding:-Padding]

    ANN_real_Ten = ANN_result_Ten.real.cpu().detach()
    ANN_imag_Ten = ANN_result_Ten.imag.cpu().detach()
    ANN_phase_Ten = torch.angle(ANN_result_Ten).cpu().detach()
    # ANN_real_Ten = torch.nn.functional.interpolate(ANN_real_Ten.unsqueeze(0).unsqueeze(0), scale_factor=(Dim_XY/ANN_real_Ten.size(0),Dim_XY/ANN_real_Ten.size(1)), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
    # ANN_imag_Ten = torch.nn.functional.interpolate(ANN_imag_Ten.unsqueeze(0).unsqueeze(0), scale_factor=(Dim_XY/ANN_imag_Ten.size(0),Dim_XY/ANN_imag_Ten.size(1)), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)


    ANN_real_Arr = ANN_real_Ten.numpy()
    ANN_imag_Arr = ANN_imag_Ten.numpy()
    ANN_phase_Arr = ANN_phase_Ten.numpy()

    ANN_f_phase_Arr = torch.angle(ANN_f_result_Ten).cpu().detach().numpy()

    if Padding > 0:
        nopad_phase_Arr = phase_Ten_no_pad.cpu().detach().numpy()
        nopad_real_Arr = real_Arr[Padding:-Padding, Padding:-Padding]
        nopad_imag_Arr = imag_Arr[Padding:-Padding, Padding:-Padding]
    else: 
        nopad_phase_Arr = phase_Ten_no_pad.cpu().detach().numpy()
        nopad_real_Arr = real_Arr
        nopad_imag_Arr = imag_Arr

    ANN_f_all_Ten = fftn_shift(torch.from_numpy(ANN_real_Arr + ANN_imag_Arr*1j))
    ANN_nopad_f_phase_Arr = torch.angle(ANN_f_all_Ten).numpy()

    f_phase_Arr = f_phase_Ten.cpu().detach().numpy()
    nopad_f_phase_Arr = f_phase_Ten_no_pad.cpu().detach().numpy()

In [ ]:
#@title Функция вывода
#@markdown
Delta_Range = ()
Delta_Range = list(Delta_Range)

norm_const = 1
    
def graph_print():
    right_phase_shift = 0
    min_mean = float('inf')
    
    # for phase_shift in range(4):
    #     init_graphs(phase_shift)
    #     mean = np.mean(ring_alg(ANN_f_phase_Arr, nopad_f_phase_Arr)).item(0)/math.pi*100
    #     if min_mean > mean:
    #         right_phase_shift = phase_shift
    #         min_mean = mean
    # init_graphs(right_phase_shift)
    init_graphs(0)
        
    # Выводим параметры
    # Data_Str = 'Картинки: ' + str(pics_Name) + '\nРазмер: ' + str(Dim_XY) + '\nПаддинг: ' + str(Padding)
    Data_Str = str(pics_Name) + ';  Размер: ' + str(Dim_XY) + ';  Паддинг: ' + str(Padding) + ';  Эпохи: ' + str(Train_Sum_Epochs//10*10) + ';  Время: ' + str(round(Train_Sum_Time))+' сек'
    Data_fig, Data_ax = plt.subplots(figsize=(26,1))
    Data_ax.get_xaxis().set_visible(False)
    Data_ax.get_yaxis().set_visible(False)
    Data_fig.text(0.2, 0.5, Data_Str, ha='left', va='center', size=35)
    plt.show()
    print() 


    # Выводим действ. картинки
    ANN_fig_real, ANN_ax_real = plt.subplots(1,3, figsize=(28,6))
    ANN_fig_real.suptitle('Real part', fontsize=24)

    plot_image(np.abs(ANN_real_Arr), ANN_ax_real[0], cmap='gray', title="Result")
    plot_image(np.flip(np.abs(ANN_real_Arr), (0, 1)), ANN_ax_real[1], cmap='gray', title="Result 180 гр")
    plot_image(nopad_real_Arr, ANN_ax_real[2], cmap='gray', title="Original")
    plt.show()
    print() 

    # # Выводим действ. картинки
    # ANN_fig_real, ANN_ax_real = plt.subplots(1,3, figsize=(28,6))
    # ANN_fig_real.suptitle('Real part', fontsize=24)

    # plot_image(np.abs(ANN_real_Arr), ANN_ax_real[0], cmap='gray', title="Result")
    # plot_image(nopad_real_Arr, ANN_ax_real[1], cmap='gray', title="Original")
    # plot_image(np.abs(np.abs(ANN_real_Arr)-nopad_real_Arr), ANN_ax_real[2], cmap='gray', title="Delta")
    # plt.show()

    # # Выводим действ. картинки с поворотом
    # ANN_fig_real, ANN_ax_real = plt.subplots(1,3, figsize=(28,6))
    # ANN_fig_real.suptitle('Real part', fontsize=24)

    # plot_image(np.flip(np.abs(ANN_real_Arr), (0, 1)), ANN_ax_real[0], cmap='gray', title="Result 180 гр")
    # plot_image(np.flip(nopad_real_Arr, (0, 1)), ANN_ax_real[1], cmap='gray', title="Original 180 гр")
    # plot_image(np.flip(np.abs(np.abs(ANN_real_Arr)-nopad_real_Arr), (0, 1)), ANN_ax_real[2], cmap='gray', title="Delta")
    # plt.show()

#     # Выводим "отбивку"
#     Data_fig, Data_ax = plt.subplots(figsize=(28,0.5))
#     Data_ax.get_xaxis().set_visible(False)
#     Data_ax.get_yaxis().set_visible(False)
#     plt.show()

    # # Выводим мним. картинки
    # ANN_fig_imag, ANN_ax_imag = plt.subplots(1,3, figsize=(28,6))
    # ANN_fig_imag.suptitle('Imaginary part', fontsize=24)
    # plot_image(np.abs(ANN_imag_Arr), ANN_ax_imag[0], cmap='gray', title="Result", vmin=0)
    # plot_image(nopad_imag_Arr, ANN_ax_imag[1], cmap='gray', title="Original", vmin=0, vmax=1)
    # plot_image(np.abs(np.abs(ANN_imag_Arr)-nopad_imag_Arr), ANN_ax_imag[2], cmap='gray', title="Delta", vmin=0)

    # # Выводим "отбивку"
    # Data_fig, Data_ax = plt.subplots(figsize=(28,0.5))
    # Data_ax.get_xaxis().set_visible(False)
    # Data_ax.get_yaxis().set_visible(False)
    # plt.show()


    # # Выводим фазы прямого изображения
    # ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,3, figsize=(28,6))
    # ANN_fig_f_phase.suptitle('Direct Phases', fontsize=24)
    # plot_image(ANN_phase_Arr, ANN_ax_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
    # plot_image(nopad_phase_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
    # plot_image(ring_alg(ANN_phase_Arr, nopad_phase_Arr), ANN_ax_f_phase[2], cmap='gray', title="Delta")

    # # Выводим амплитуды фурье-образа
    # ANN_fig_f_ampl, ANN_ax_ampl = plt.subplots(1,3, figsize=(28,6))
    # ANN_fig_f_ampl.suptitle('Fourie Transform Ampls', fontsize=24)
    # plot_image(ANN_f_ampl_Arr, ANN_ax_ampl[0], title="Result", logscale=True)
    # plot_image(f_ampl_Ten.detach().cpu().numpy(), ANN_ax_ampl[1], title="Original", logscale=True)
    # plot_image(ring_alg(ANN_f_ampl_Arr, f_ampl_Ten.detach().cpu().numpy()), ANN_ax_ampl[2], title="Delta")

    # # Выводим "отбивку"
    # Data_fig, Data_ax = plt.subplots(figsize=(28,0.5))
    # Data_ax.get_xaxis().set_visible(False)
    # Data_ax.get_yaxis().set_visible(False)
    # plt.show()

    # Выводим фазы фурье-образа картинок
    ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,3, figsize=(28,6))
    ANN_fig_f_phase.suptitle('Fourie Transform Phases', fontsize=24)
    plot_image(ANN_f_phase_Arr, ANN_ax_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
    plot_image(f_phase_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
    plot_image(ring_alg(ANN_f_phase_Arr, f_phase_Arr), ANN_ax_f_phase[2], cmap='gray', title="Delta", vmin=0, vmax=math.pi)
    plt.show()
    print()

    # Выводим фазы фурье-образа картинок без паддинга
    ANN_fig_f_nopad_phase, ANN_ax_nopad_f_phase = plt.subplots(1,3, figsize=(28,6))
    ANN_fig_f_nopad_phase.suptitle('Fourie Transform Phases No Padding', fontsize=24)
    plot_image(ANN_nopad_f_phase_Arr, ANN_ax_nopad_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
    plot_image(nopad_f_phase_Arr, ANN_ax_nopad_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
    plot_image(ring_alg(ANN_nopad_f_phase_Arr, nopad_f_phase_Arr), ANN_ax_nopad_f_phase[2], cmap='gray', title="Delta", vmin=0, vmax=math.pi)
    plt.show()
    print() 

    Delta_Range.append(round(np.mean(ring_alg(ANN_f_phase_Arr, f_phase_Arr)).item(0)/math.pi*100))

In [ ]:
graph_print()

In [ ]:
print(Delta_Range)

In [ ]:
def random_start(amplitudes): # Генерируем T1* по случайным фазам 
    random_phase = math.pi * (torch.rand(amplitudes.size()) * 2 - 1)
    return torch.polar(amplitudes.to(device), random_phase.to(device))


def iter_mask(real_data, real_mask): # Создаем маску иттерации - помечаем всё прямое пространство < 0 и паддинги
    return torch.logical_or(real_data<0, real_mask) 


def real_support_er(real_data, real_mask): # Зануляем тензор по маске
    new_real_data = real_data.clone()
    new_real_data[real_mask] = 0
    return new_real_data


def real_support_hio(real_data, real_mask, prev_real_data, beta):
    new_real_data = real_data.clone()
    new_real_data[real_mask] = prev_real_data[real_mask] - (1-beta) * new_real_data[real_mask]
    return new_real_data


def real_error(real_data, real_mask):
    return torch.sum(torch.abs(real_data[real_mask]) ** 2)/torch.sum(torch.abs(real_data[real_mask == 0]) ** 2)


def recip_support(recip_data, amplitudes): # Применяем угаданные на итерации фазы к реальным амплитудам
    phases = torch.angle(recip_data)
    return torch.polar(amplitudes, phases)

def recip_error(recip_data, amplitudes):
    return torch.sqrt(torch.sum(torch.abs(recip_data) - amplitudes) ** 2 / torch.sum(amplitudes ** 2))
    

In [ ]:
T1_c_pri = random_start(f_ampl_Ten)
T1_c_pri = torch.from_numpy(ANN_train_res_PhaseSolver_init_Arr)
T1_pri = torch.abs(T1_c_pri)
TLenna_c_pri = ifftn_shift(T1_c_pri)
TLenna_pri = torch.abs(TLenna_c_pri)
real_result_sup = real_support_er(TLenna_pri, mask_Ten)

fig, ax = plt.subplots(figsize=(6,6))
plot_image(real_result_sup.real.cpu().detach().numpy(), ax, logscale=False, cmap='gray')

In [ ]:
from re import X
# T1_c_pri = random_start(f_ampl_Ten).to(device)
T1_c_pri = torch.polar(f_ampl_Ten,torch.from_numpy(ANN_train_res_PhaseSolver_init_Arr).to(device))

n_hio = 400
n_er = 0
n_repeat = 1

n_total = n_repeat*(n_hio + n_er)

beta_start = 0.1
beta_end = 0.1

real_error_vals = np.zeros(n_total)
recip_error_vals = np.zeros(n_total)

report_step = n_total//6
real_report = []
recip_report = []
PCA_HIO = []
PCA_HIO_LOSS = []

prev_TLenna_c_pri = ifftn_shift(T1_c_pri)
x = prev_TLenna_c_pri
x_prev = x

PCA_HIO = []
PCA_HIO_LOSS = []

PCA_HIO.append(fftn_shift(x).angle().detach().cpu().numpy())
loss, loss_cheat, loss_pad, loss_d_phas, loss_mirror, loss_nega, loss_imag = PS_Loss(fftn_shift(x).angle())
PCA_HIO_LOSS.append(loss_cheat.detach().item())

Train_Sum_Time_HIO = 0

for i in range(n_total):
    
    Epoch_Time = datetime.datetime.now()
    hio = (i % (n_hio + n_er)) < n_hio
    beta = beta_start + (beta_end - beta_start)*i/n_total

    new_T1_c_pri = fftn_shift(x)
    recip_error_vals[i] = recip_error(new_T1_c_pri, f_ampl_Ten).detach().item()

    new_T1_c_pri_phase = new_T1_c_pri.angle()
    if MSE(ifftn_shift(torch.polar(f_ampl_Ten, new_T1_c_pri_phase)).real, ampl_Ten) > MSE(ifftn_shift(torch.polar(f_ampl_Ten, -new_T1_c_pri_phase)).real, ampl_Ten):
        new_T1_c_pri_phase = -new_T1_c_pri_phase
    
    PCA_HIO.append(new_T1_c_pri.angle().detach().cpu().numpy())
    loss, loss_cheat, loss_pad, loss_d_phas, loss_mirror, loss_nega, loss_imag = PS_Loss(new_T1_c_pri.angle())
    PCA_HIO_LOSS.append(loss_cheat.detach().item())
    
    new_T1_c_res = torch.polar(f_ampl_Ten, new_T1_c_pri_phase)
    TLenna_c_pri = ifftn_shift(new_T1_c_res)
    TLenna_pri = TLenna_c_pri.real
    it_mask = iter_mask(TLenna_pri, mask_Ten)

    
#     PCA_HIO.append(fftn_shift(TLenna_pri).angle().detach().cpu().numpy())
#     loss, loss_cheat, loss_pad, loss_d_phas, loss_mirror, loss_nega, loss_imag = PS_Loss(fftn_shift(TLenna_pri).angle())
#     PCA_HIO_LOSS.append(loss_cheat.detach().item())

    if hio:
        TLenna_c_res = real_support_hio(TLenna_c_pri, it_mask, x, beta)
    else:
        TLenna_c_res = real_support_er(TLenna_c_pri, it_mask)

    real_error_vals[i] = real_error(TLenna_c_res, it_mask).detach().item()
    
    x = TLenna_c_res
    delta_Time = datetime.datetime.now() - Epoch_Time
    Train_Sum_Time_HIO += (datetime.datetime.now() - Epoch_Time).total_seconds()

    print('%-4s' % (str(int(i/epochs_Num*1000))+'‰'),
                '- Ep: '+ '%-5s'%str(i//10*10), 
                '- LSum: '+'%10s'%('%.6f'%loss.item()), 
                # '- LPha: '+'%10s'%('%.6f'%loss_phase.item()), 
                # '- LIma: '+'%10s'%('%.6f'%loss_imag.item()), 
                '- LD_CHEAT: '+'%10s'%('%.6f'%loss_cheat.item()), 
                # '- LDPhas: '+'%10s'%('%.6f'%loss_d_phas.item()), 
                '- LNega: '+'%10s'%('%.6f'%loss_nega.item()), 
                '- LImag: '+'%10s'%('%.6f'%loss_imag.item()), 
                '- LPad: '+'%10s'%('%.6f'%loss_pad.item()),
                '- Sec: '+'%4s'%'%.2f'%round(Train_Sum_Time_HIO,2)
          )

    if (i % 10 == 0):
      ANN_f_phas_Arr = fftn_shift(x.real).angle().detach().cpu().numpy()
      ANN_real_Arr = x.real.detach().cpu().numpy()
      ANN_f_phas_Arr_Delta = ANN_f_phas_Arr-fftn_shift(x_prev.real).angle().detach().cpu().numpy()
      ANN_f_phas_Arr_Delta_20 = ANN_f_phas_Arr_Delta.copy()
      ANN_f_phas_Arr_Delta_20[abs(ANN_f_phas_Arr_Delta)<abs(ANN_f_phas_Arr_Delta).max()*0.8] = 0

      # Выводим фазы фурье-образа картинок
      ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,4, figsize=(28,6), subplot_kw={"projection": "3d"})
      ANN_fig_f_phase.suptitle('Fourie Transform Phases (HIO+ERA)', fontsize=24)
      plot_image_3D(f_phase_Ten.detach().cpu().numpy(), ANN_ax_f_phase[0], cmap='twilight', title="Оригинал")
      plot_image_3D(ANN_f_phas_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Приближение(k)")
      plot_image_3D(ANN_f_phas_Arr_Delta, ANN_ax_f_phase[2], cmap='twilight', title="Разница между k и k-10", alpha=0.5)
      plot_image_3D(ANN_f_phas_Arr_Delta_20, ANN_ax_f_phase[3], cmap='twilight', title="Разница между k и k-10 (20% от max)", alpha=0.5)
      plt.show()
      print()
      # Выводим фазы фурье-образа картинок
      ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,4, figsize=(28,6))
      ANN_fig_f_phase.suptitle('Fourie Transform Phases (HIO+ERA)', fontsize=24)
      plot_image(ANN_real_Arr, ANN_ax_f_phase[0], cmap='gray', title="Приближение(k) I")
      plot_image(ANN_f_phas_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Приближение(k)")
      plot_image(ANN_f_phas_Arr_Delta, ANN_ax_f_phase[2], cmap='twilight', title="Разница между k и k-10")
      plot_image(ANN_f_phas_Arr_Delta_20, ANN_ax_f_phase[3], cmap='twilight', title="Разница между k и k-10 (20% от max)")
      plt.show()
      print()
      
      x_prev = x
    # print('{} r:{:2.4f} f:{:2.4f}'.format(i, real_error_vals[i], recip_error_vals[i]))
    if ((i+1)%report_step == 0):
        real_report.append(TLenna_c_pri.cpu().detach().numpy())
        recip_report.append(T1_c_pri.cpu().detach().numpy())


In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(14,8))
for i, axis in enumerate(ax.flatten()):
    plot_image(np.abs(real_report[i]), axis, logscale=False, cmap='gray')
    # axis.set_xlim([5, Dim_XY-5])
    # axis.set_ylim([100, 356])

loss, loss_cheat, loss_pad, loss_d_phas, loss_mirror, loss_nega, loss_imag = PS_Loss(fftn_shift(torch.from_numpy(real_report[-1]).abs().to(device)).angle())
print(
                '- LSum: '+'%10s'%('%.6f'%loss.item()), 
                '- LDPhas: '+'%10s'%('%.6f'%loss_d_phas.item()), 
                '- LPad: '+'%10s'%('%.6f'%loss_pad.item()), 
)

In [ ]:
HIO_real_Ten = ifftn_shift(torch.polar(f_ampl_Ten, fftn_shift(torch.from_numpy(real_report[-1]).to(device)).angle())).real
HIO_real_Ten = torch.flip(HIO_real_Ten, [0,1])

HIO_real_Arr = HIO_real_Ten[Padding:-Padding,Padding:-Padding].real.detach().cpu().numpy()
HIO_f_phase_Arr = fftn_shift(HIO_real_Ten).angle().detach().cpu().numpy()
if Padding > 0:
  HIO_nopad_f_phase_Arr = fftn_shift(HIO_real_Ten[Padding:-Padding,Padding:-Padding]).angle().detach().cpu().numpy()
else:
  HIO_nopad_f_phase_Arr = HIO_f_phase_Arr


# Выводим действ. картинки
ANN_fig_real, ANN_ax_real = plt.subplots(1,3, figsize=(28,6))
ANN_fig_real.suptitle('Real part', fontsize=24)

plot_image(np.abs(HIO_real_Arr), ANN_ax_real[0], cmap='gray', title="Result")
plot_image(np.flip(np.abs(HIO_real_Arr), (0, 1)), ANN_ax_real[1], cmap='gray', title="Result 180 гр")
plot_image(nopad_real_Arr, ANN_ax_real[2], cmap='gray', title="Original")
plt.show()
print() 

# Выводим фазы фурье-образа картинок
ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,3, figsize=(28,6), subplot_kw={"projection": "3d"})
# ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,3, figsize=(28,6))
ANN_fig_f_phase.suptitle('Fourie Transform Phases', fontsize=24)
plot_image_3D(HIO_f_phase_Arr, ANN_ax_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
plot_image_3D(f_phase_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
plot_image_3D(ring_alg(HIO_f_phase_Arr, f_phase_Arr), ANN_ax_f_phase[2], cmap='gray', title="Delta", vmin=0, vmax=math.pi)
# plot_image(HIO_f_phase_Arr, ANN_ax_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
# plot_image(f_phase_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
# plot_image(ring_alg(HIO_f_phase_Arr, f_phase_Arr), ANN_ax_f_phase[2], cmap='gray', title="Delta", vmin=0, vmax=math.pi)
plt.show()
print()

# Выводим фазы фурье-образа картинок без паддинга
ANN_fig_f_nopad_phase, ANN_ax_nopad_f_phase = plt.subplots(1,3, figsize=(28,6), subplot_kw={"projection": "3d"})
# ANN_fig_f_nopad_phase, ANN_ax_nopad_f_phase = plt.subplots(1,3, figsize=(28,6))
ANN_fig_f_nopad_phase.suptitle('Fourie Transform Phases No Padding', fontsize=24)
plot_image_3D(HIO_nopad_f_phase_Arr, ANN_ax_nopad_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
plot_image_3D(nopad_f_phase_Arr, ANN_ax_nopad_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
plot_image_3D(ring_alg(HIO_nopad_f_phase_Arr, nopad_f_phase_Arr), ANN_ax_nopad_f_phase[2], cmap='gray', title="Delta", vmin=0, vmax=math.pi)
# plot_image(HIO_nopad_f_phase_Arr, ANN_ax_nopad_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
# plot_image(nopad_f_phase_Arr, ANN_ax_nopad_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
# plot_image(ring_alg(HIO_nopad_f_phase_Arr, nopad_f_phase_Arr), ANN_ax_nopad_f_phase[2], cmap='gray', title="Delta", vmin=0, vmax=math.pi)
plt.show()
print() 

In [ ]:
## Выводим действ. картинки
# k = 3
# for i in range(1, 15):
#   crop_size = int(i*k)
#   TEST_f_phase_Ten = f_phase_Ten[crop_size:-crop_size, crop_size:-crop_size]
#   TEST_f_phase_Ten = torch.nn.functional.pad(TEST_f_phase_Ten, (crop_size,crop_size, crop_size,crop_size))
#   TEST_real_Ten = ifftn_shift(torch.polar(f_ampl_Ten, TEST_f_phase_Ten)).real
#   TEST_real_Arr = TEST_real_Ten.detach().cpu().numpy()

#   ANN_fig_real, ANN_ax_real = plt.subplots(1,3, figsize=(28,6))
#   ANN_fig_real.suptitle('Real part', fontsize=24)
#   plot_image(TEST_f_phase_Ten.detach().cpu().numpy(), ANN_ax_real[0], cmap='twilight', title=str(i*k)+' вырезано по краю')
#   plot_image(np.abs(TEST_real_Arr), ANN_ax_real[1], cmap='gray', title=str(i*k)+' вырезано по краю')
#   plot_image(real_Ten.detach().cpu().numpy(), ANN_ax_real[2], cmap='gray', title="Оригинал")
#   plt.show()
#   print() 

In [ ]:
phase_n = 150
PCA_HIO_ = np.stack(PCA_HIO[::len(PCA_HIO)//phase_n][-phase_n:])
PCA_HIO_LOSS_ = np.stack(PCA_HIO_LOSS[::len(PCA_HIO_LOSS)//phase_n][-phase_n:])
print(PCA_HIO_.shape)

print(MSE(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), torch.from_numpy(PCA_HIO_[-1]))).real*255, ampl_Ten.detach().cpu()*255), PCA_HIO_LOSS_[-1])

In [ ]:
pca_hio = PCA(n_components=2)
PCA_HIO_ANS = pca_hio.fit_transform(PCA_HIO_.reshape(PCA_HIO_.shape[0], -1))
# PCA_HIO_ANS = np.array(PCA_HIO_ANS)
print(PCA_HIO_ANS.shape)

In [ ]:
HIO_c = np.stack([[0.0,0.0,1.0]]*PCA_HIO_ANS.shape[0])
HIO_c[:,2] = np.log(1 + HIO_c[:,2]*(1-(PCA_HIO_LOSS_-PCA_HIO_LOSS_.min())/(PCA_HIO_LOSS_.max()-PCA_HIO_LOSS_.min())))
plt.scatter(PCA_HIO_ANS[:, 0], PCA_HIO_ANS[:, 1], c=HIO_c)
plt.gca().set(
    aspect="equal",
    title="HIO",
    xlabel="first feature",
    ylabel="second feature",
)
plt.show()


In [ ]:
PCA_ALL_ = np.concatenate((PCA_ANN_, PCA_ANN_HIO_, PCA_HIO_, ANN_train_res_PhaseSolver_init_Arr.reshape(1,Dim_XY_P,Dim_XY_P), f_phase_Ten.detach().cpu().numpy().reshape(1,Dim_XY_P,Dim_XY_P)))
print(PCA_ALL_.shape)
print(PCA_ALL_[-2]) 
print(PCA_ALL_[-1]) 
# pca_all = PCA(n_components=2)
def angle_metric(X1,X2):
  X = np.square(
          np.abs(
              np.subtract(
                np.exp(X1*1j), np.exp(X2*1j)
              )
          )
      ).mean()
  return X

pca_dim_n = 2

pca_all = TSNE(n_components=pca_dim_n, metric=angle_metric)
PCA_ALL_ANS = pca_all.fit_transform(PCA_ALL_.reshape(PCA_ALL_.shape[0], -1))

In [ ]:
torch.save(torch.from_numpy(PCA_ALL_ANS), 'ALL.pt')

In [ ]:
PCA_ALL_ANS = torch.load('ALL.pt').numpy()
# PCA_HIO_ANS = np.array(PCA_HIO_ANS)
print(PCA_ALL_ANS.shape)

PCA_ALL_ANS_ANN = PCA_ALL_ANS[0:phase_n]
PCA_ALL_ANS_ANN_HIO = PCA_ALL_ANS[phase_n:phase_n*2]
PCA_ALL_ANS_HIO = PCA_ALL_ANS[phase_n*2:-2]

In [ ]:
# print(PCA_ALL_ANS_ANN.shape,PCA_ALL_ANS_HIO.shape)

# ANN_fig_real, ANN_ax_real = plt.subplots(3, figsize=(14,20))

# ANN_fig_real.suptitle("ORIG", fontsize=20)

# ANN_ax_real[0].scatter(PCA_ALL_ANS_ANN[:,0], PCA_ALL_ANS_ANN[:,1], c=ANN_c, label="ANN")
# ANN_ax_real[0].set(
#     # aspect="equal",
#     title="ANN",
#     xlabel="first feature",
#     ylabel="second feature",
# )
# ANN_ax_real[0].legend()

# ANN_ax_real[1].scatter(PCA_ALL_ANS_HIO[:,0], PCA_ALL_ANS_HIO[:,1], c=HIO_c, label="HIO")
# ANN_ax_real[1].set(
#     # aspect="equal",
#     title="HIO",
#     xlabel="first feature",
#     ylabel="second feature",
# )
# ANN_ax_real[1].legend()

# ANN_ax_real[2].scatter(PCA_ALL_ANS_ANN[:,0], PCA_ALL_ANS_ANN[:,1], c=ANN_c, label="ANN")
# ANN_ax_real[2].scatter(PCA_ALL_ANS_HIO[:,0], PCA_ALL_ANS_HIO[:,1], c=HIO_c, label="HIO")
# ANN_ax_real[2].set(
#     # aspect="equal",
#     title="ANN+HIO",
#     xlabel="first feature",
#     ylabel="second feature",
# )
# ANN_ax_real[2].legend()

# plt.show()

In [ ]:
# PCA_ALL_ANS_ANN = PCA_ALL_ANS_ANN/abs(PCA_ALL_ANS_ANN.max())
# PCA_ALL_ANS_HIO = PCA_ALL_ANS_HIO/abs(PCA_ALL_ANS_HIO.max())
# PCA_ALL_ANS_ANN = PCA_ALL_ANS_ANN-PCA_ALL_ANS_ANN[0]
# PCA_ALL_ANS_HIO = PCA_ALL_ANS_HIO-PCA_ALL_ANS_HIO[0]

# ANN_fig_real, ANN_ax_real = plt.subplots(3, figsize=(14,20))

# ANN_fig_real.suptitle("NORM", fontsize=20)

# ANN_ax_real[0].scatter(PCA_ALL_ANS_ANN[:,0], PCA_ALL_ANS_ANN[:,1], c=ANN_c, label="ANN")
# ANN_ax_real[0].set(
#     # aspect="equal",
#     title="ANN",
#     xlabel="first feature",
#     ylabel="second feature",
# )
# ANN_ax_real[0].legend()

# ANN_ax_real[1].scatter(PCA_ALL_ANS_HIO[:,0], PCA_ALL_ANS_HIO[:,1], c=HIO_c, label="HIO")
# ANN_ax_real[1].set(
#     # aspect="equal",
#     title="HIO",
#     xlabel="first feature",
#     ylabel="second feature",
# )
# ANN_ax_real[1].legend()

# ANN_ax_real[2].scatter(PCA_ALL_ANS_ANN[:,0], PCA_ALL_ANS_ANN[:,1], c=ANN_c, label="ANN")
# ANN_ax_real[2].scatter(PCA_ALL_ANS_HIO[:,0], PCA_ALL_ANS_HIO[:,1], c=HIO_c, label="HIO")
# ANN_ax_real[2].set(
#     # aspect="equal",
#     title="ANN+HIO",
#     xlabel="first feature",
#     ylabel="second feature",
# )
# ANN_ax_real[2].legend()

# plt.show()

In [ ]:
# PCA_ALL_ANS_HIO = PCA_ALL_ANS[phase_n:-2]

# n_grid = 100
# PCA_HIO_GRID_X, PCA_HIO_GRID_Y  =  np.meshgrid(np.linspace(PCA_ALL_ANS_HIO[:,0].min(), PCA_ALL_ANS_HIO[:,0].max(), n_grid), 
#                                                np.linspace(PCA_ALL_ANS_HIO[:,1].min(), PCA_ALL_ANS_HIO[:,1].max(), n_grid))
# PCA_HIO_GRID = np.stack((PCA_HIO_GRID_X, PCA_HIO_GRID_Y), -1)
# PCA_HIO_GRID_PHASE = pca_hio.inverse_transform(PCA_HIO_GRID).reshape(-1,Dim_XY_P,Dim_XY_P)


# PCA_HIO_GRID_LOSS = np.zeros(PCA_HIO_GRID_PHASE.shape[0])
# for i in range(PCA_HIO_GRID_PHASE.shape[0]):
#   l_, a, a, a, a = PS_Loss(torch.from_numpy(PCA_HIO_GRID_PHASE[i]).to(device).to(torch.float))
#   PCA_HIO_GRID_LOSS[i] = l_.detach().cpu().numpy()
# print(PCA_HIO_GRID_LOSS.shape)
# print(PCA_HIO_GRID_LOSS)

# HIO_GRID_c = np.stack([[1.0,1.0,1.0]]*PCA_HIO_GRID_LOSS.shape[0])
# HIO_GRID_c[:,0] = np.log(1 + HIO_GRID_c[:,0]*(PCA_HIO_GRID_LOSS/PCA_HIO_GRID_LOSS.max()))
# HIO_GRID_c[:,1] = np.log(1 + HIO_GRID_c[:,1]*(PCA_HIO_GRID_LOSS/PCA_HIO_GRID_LOSS.max()))

In [ ]:
# PCA_ALL_ANS_ANN = PCA_ALL_ANS[0:phase_n]

# n_grid = 100
# PCA_ANN_GRID_X, PCA_ANN_GRID_Y  =  np.meshgrid(np.linspace(PCA_ALL_ANS_ANN[:,0].min(), PCA_ALL_ANS_ANN[:,0].max(), n_grid), 
#                                                np.linspace(PCA_ALL_ANS_ANN[:,1].min(), PCA_ALL_ANS_ANN[:,1].max(), n_grid))
# PCA_ANN_GRID = np.stack((PCA_ANN_GRID_X, PCA_ANN_GRID_Y), -1)
# PCA_ANN_GRID_PHASE = pca_ann.inverse_transform(PCA_ANN_GRID).reshape(-1,Dim_XY_P,Dim_XY_P)


# PCA_ANN_GRID_LOSS = np.zeros(PCA_ANN_GRID_PHASE.shape[0])
# for i in range(PCA_ANN_GRID_PHASE.shape[0]):
#   l_, a, a, a, a = PS_Loss(torch.from_numpy(PCA_ANN_GRID_PHASE[i]).to(device).to(torch.float))
#   PCA_ANN_GRID_LOSS[i] = l_.detach().cpu().numpy()
# print(PCA_ANN_GRID_LOSS.shape)
# print(PCA_ANN_GRID_LOSS)

# ANN_GRID_c = np.stack([[1.0,0,0]]*PCA_ANN_GRID_LOSS.shape[0])
# ANN_GRID_c[:,0] = np.log(1 + ANN_GRID_c[:,0]*(1-PCA_ANN_GRID_LOSS/PCA_ANN_GRID_LOSS.max()))

In [ ]:
# n_grid = 100
# PCA_ALL_GRID_X, PCA_ALL_GRID_Y  =  np.meshgrid(np.linspace(PCA_ALL_ANS[:,0].min(), PCA_ALL_ANS[:,0].max(), n_grid), 
#                                                np.linspace(PCA_ALL_ANS[:,1].min(), PCA_ALL_ANS[:,1].max(), n_grid))
# PCA_ALL_GRID = np.stack((PCA_ALL_GRID_X, PCA_ALL_GRID_Y), -1)
# PCA_ALL_GRID_PHASE = pca_all.inverse_transform(PCA_ALL_GRID).reshape(-1,Dim_XY_P,Dim_XY_P)


# PCA_ALL_GRID_LOSS = np.zeros(PCA_ALL_GRID_PHASE.shape[0])
# for i in range(PCA_ALL_GRID_PHASE.shape[0]):
#   l_, a, a, a, a = PS_Loss(torch.from_numpy(PCA_ALL_GRID_PHASE[i]).to(device).to(torch.float))
#   PCA_ALL_GRID_LOSS[i] = l_.detach().cpu().numpy()
# print(PCA_ALL_GRID_LOSS.shape)
# print(PCA_ALL_GRID_LOSS)

# ALL_GRID_c = np.stack([[1.0,1.0,1.0]]*PCA_ALL_GRID_LOSS.shape[0])
# ALL_GRID_c[:,0] = np.log(1 + ALL_GRID_c[:,0]*(PCA_ALL_GRID_LOSS/PCA_ALL_GRID_LOSS.max()))
# ALL_GRID_c[:,2] = np.log(1 + ALL_GRID_c[:,2]*(PCA_ALL_GRID_LOSS/PCA_ALL_GRID_LOSS.max()))

In [ ]:
# f_grid, ax_grid = plt.subplots(2, figsize=(14,20))

# ANN_fig_real.suptitle("ORIG", fontsize=20)

# ax_grid[0].scatter(PCA_HIO_GRID_X, PCA_HIO_GRID_Y, c=HIO_GRID_c)
# ax_grid[0].scatter(PCA_ALL_ANS_HIO[:,0], PCA_ALL_ANS_HIO[:,1], c=HIO_c, label="HIO")
# ax_grid[0].set(
#     title="HIO",
#     xlabel="first feature",
#     ylabel="second feature",
# )


# ax_grid[1].scatter(PCA_ANN_GRID_X, PCA_ANN_GRID_Y, c=ANN_GRID_c)
# ax_grid[1].scatter(PCA_ALL_ANS_ANN[:,0], PCA_ALL_ANS_ANN[:,1], c=ANN_c, label="ANN")
# ax_grid[1].set(
#     title="ANN",
#     xlabel="first feature",
#     ylabel="second feature",
# )
# plt.show()

In [ ]:
PCA_ALL_ANS_ANN = PCA_ALL_ANS[0:phase_n]
PCA_ALL_ANS_ANN_HIO = PCA_ALL_ANS[phase_n:phase_n*2]
PCA_ALL_ANS_HIO = PCA_ALL_ANS[phase_n*2:-2]

f_grid = plt.figure(1, figsize=(14,20))
if pca_dim_n == 3:
  ax_grid = f_grid.add_subplot(projection='3d')
elif pca_dim_n == 2:
  ax_grid = f_grid.add_subplot()


ANN_fig_real.suptitle("ORIG", fontsize=20)

# ax_grid.scatter(PCA_ALL_GRID_X, PCA_ALL_GRID_Y, c=ALL_GRID_c)
ax_grid.scatter(PCA_ALL_ANS_HIO[:,0], PCA_ALL_ANS_HIO[:,1], c=HIO_c, label="HIO")
ax_grid.scatter(PCA_ALL_ANS_ANN[:,0], PCA_ALL_ANS_ANN[:,1], c=ANN_c, label="SGD")
ax_grid.scatter(PCA_ALL_ANS_ANN_HIO[:,0], PCA_ALL_ANS_ANN_HIO[:,1], c=ANN_HIO_c, label="SGD+HIO")
ax_grid.scatter(PCA_ALL_ANS_HIO[0,0], PCA_ALL_ANS_HIO[0,1], c='cyan', label="HIO", marker='^')
ax_grid.scatter(PCA_ALL_ANS_ANN[0,0], PCA_ALL_ANS_ANN[0,1], c="orange", label="SGD", marker='^')
ax_grid.scatter(PCA_ALL_ANS_ANN_HIO[0,0], PCA_ALL_ANS_ANN_HIO[0,1], c="lime", label="SGD+HIO", marker='^')
ax_grid.scatter(PCA_ALL_ANS[-2,0], PCA_ALL_ANS[-2,1], c='gray', label="Init", marker='^')
ax_grid.scatter(PCA_ALL_ANS[-1,0], PCA_ALL_ANS[-1,1], c='purple', label="True", marker='^')


if pca_dim_n == 3:
  ax_grid.set(
    title="HIO+SGD",
    xlabel="first feature",
    ylabel="second feature",
    zlabel="third feature",
  ) 
elif pca_dim_n == 2:
  ax_grid.set(
    title="HIO+SGD",
    xlabel="first feature",
    ylabel="second feature",
  ) 

plt.show()

In [ ]:
print('ORIG SGD:', math.sqrt(
    np.square(
        np.abs(
            np.subtract(
                np.exp(PCA_ALL_[phase_n]*1j), np.exp(PCA_ALL_[-1]*1j)
            )
        )
    ).mean())
)
print('ORIG HIO:', math.sqrt(
    np.square(
        np.abs(
            np.subtract(
                np.exp(PCA_ALL_[-phase_n]*1j), np.exp(PCA_ALL_[-1]*1j)
            )
        )
    ).mean())
)


# print('INVERSE ANN:', math.sqrt(
#     np.square(
#         np.abs(
#             np.subtract(
#               np.exp(pca_all.inverse_transform(PCA_ALL_ANS_ANN[-1]).reshape(-1,Dim_XY_P,Dim_XY_P)*1j), np.exp(PCA_ALL_[-1]*1j)
#             )
#         )
#     ).mean())
# )
# print('INVERSE HIO:', math.sqrt(
#     np.square(
#         np.abs(
#             np.subtract(
#               np.exp(pca_all.inverse_transform(PCA_ALL_ANS_HIO[-1]).reshape(-1,Dim_XY_P,Dim_XY_P)*1j), np.exp(PCA_ALL_[-1]*1j)
#             )
#         )
#     ).mean())
# )
print('PCA SGD<-->GTRUTH DISTANCE:', math.sqrt(np.square(PCA_ALL_ANS_ANN[-1] - PCA_ALL_ANS[-1]).sum()))
print('PCA HIO<-->GTRUTH DISTANCE:', math.sqrt(np.square(PCA_ALL_ANS_HIO[-1] - PCA_ALL_ANS[-1]).sum()))


In [ ]:
PCA_HIO_DIST_TO_TRUTH = np.sqrt(
  np.square(
    np.sum(
      np.subtract(
          PCA_ALL_ANS_HIO, PCA_ALL_ANS[-1]
      ),
      axis=-1
    )
  )
)
PCA_ANN_HIO_DIST_TO_TRUTH = np.sqrt(
  np.square(
    np.sum(
      np.subtract(
          PCA_ALL_ANS_ANN_HIO, PCA_ALL_ANS[-1]
      ),
      axis=-1
    )
  )
)
PCA_ANN_DIST_TO_TRUTH = np.sqrt(
  np.square(
    np.sum(
      np.subtract(
          PCA_ALL_ANS_ANN, PCA_ALL_ANS[-1]
      ),
      axis=-1
    )
  )
)

fig, ax = plt.subplots(figsize=(20,7))
ax.set(
    title="HIO+SGD",
    xlabel="Итерация",
    ylabel="Расстояние до истинной фазы"
  ) 
f_nnn_ = 0
nnn_ = -1
plt.plot(PCA_HIO_DIST_TO_TRUTH[f_nnn_:nnn_], color='blue')
plt.plot(PCA_ANN_HIO_DIST_TO_TRUTH[f_nnn_:nnn_], color='green')
plt.plot(PCA_ANN_DIST_TO_TRUTH[f_nnn_:nnn_], color='red')
plt.show()
# plt.plot(PCA_ANN_LOSS_, color='orange')
plt.plot(PCA_ANN_HIO_LOSS_[f_nnn_:nnn_], color='lime')
plt.plot(PCA_HIO_LOSS_[f_nnn_:nnn_], color='cyan')
plt.show()
# plt.plot(PCA_HIO_DIST_TO_TRUTH/PCA_HIO_DIST_TO_TRUTH.max(), color='blue')
# plt.plot(PCA_HIO_LOSS_/PCA_HIO_LOSS_.max(), color='cyan')
# plt.show()
# plt.plot(PCA_ANN_DIST_TO_TRUTH/PCA_ANN_DIST_TO_TRUTH.max(), color='red')
# plt.plot(PCA_ANN_LOSS_/PCA_ANN_LOSS_.max(), color='orange')
# plt.show()

In [ ]:
ann_final, ann_hio_final, hio_final = torch.from_numpy(PCA_ANN_[-1]), torch.from_numpy(PCA_ANN_HIO_[-1]) ,torch.from_numpy(PCA_HIO_[-1])
hio_final = fftn_shift(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), hio_final.detach().cpu())).real).angle()
ann_hio_final = fftn_shift(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), ann_hio_final)).real).angle()
ann_final = fftn_shift(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), ann_final.detach().cpu())).real).angle()

print('HIO:', MSE(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), hio_final.detach().cpu())).real*255, ampl_Ten.detach().cpu()*255).item(), PCA_HIO_LOSS_[-1])
print('HIO+SGD:', MSE(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), ann_hio_final.detach().cpu())).real*255, ampl_Ten.detach().cpu()*255).item(), PCA_ANN_HIO_LOSS_[-1])
print('SGD:', MSE(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), ann_final.detach().cpu())).real*255, ampl_Ten.detach().cpu()*255).item(), PCA_ANN_LOSS_[-1])

ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(2,4, figsize=(18,6))
plot_image(ann_final.detach().cpu().numpy().real, ANN_ax_f_phase[0][0], cmap='twilight', title = "SGD")
plot_image(ann_hio_final.detach().cpu().numpy().real, ANN_ax_f_phase[0][1], cmap='twilight', title = "SGD+HIO")
plot_image(hio_final.detach().cpu().numpy().real, ANN_ax_f_phase[0][2], cmap='twilight', title = "HIO")
plot_image(f_phase_Ten.detach().cpu().numpy().real, ANN_ax_f_phase[0][3], cmap='twilight', title = "ORIG")
plot_image(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), ann_final)).detach().cpu().numpy().real, ANN_ax_f_phase[1][0], cmap='gray')
plot_image(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), ann_hio_final)).detach().cpu().numpy().real, ANN_ax_f_phase[1][1], cmap='gray')
plot_image(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), hio_final)).detach().cpu().numpy().real, ANN_ax_f_phase[1][2], cmap='gray')
plot_image(ampl_Ten.detach().cpu().numpy().real, ANN_ax_f_phase[1][3], cmap='gray')

ANN_fig_f_phase.show()

In [ ]:
for iiii in range(6,1000):

#     ANN_train_res_PhaseSolver = f_phase_Ten.clone().detach();
#     print('MSE:', MSE(ANN_train_res_PhaseSolver, f_phase_Ten).item())
#     while MSE(ANN_train_res_PhaseSolver, f_phase_Ten).item() < 2*math.pi:
#         ANN_train_res_PhaseSolver = ANN_train_res_PhaseSolver + ANN_train_res_PhaseSolver*((torch.rand_like(f_phase_Ten).to(device)-0.5)/100 * 5)
#         ANN_train_res_PhaseSolver = ANN_train_res_PhaseSolver
#     ANN_train_res_PhaseSolver = ANN_train_res_PhaseSolver%(2*math.pi)

    ANN_train_res_PhaseSolver = torch.rand_like(f_phase_Ten).to(device)*2*math.pi;
    print('MSE:', MSE(ANN_train_res_PhaseSolver, f_phase_Ten).item())
    ANN_train_res_PhaseSolver_init_Arr = ANN_train_res_PhaseSolver.clone().detach().cpu().numpy()
    ANN_train_res_PhaseSolver.requires_grad_()

    ANN_fig_real, ANN_ax_real = plt.subplots(2,1,figsize=(14,6))
    plot_image(ANN_train_res_PhaseSolver_init_Arr, ANN_ax_real[0], cmap='twilight')
    plot_image(ifftn_shift(torch.polar(f_ampl_Ten,ANN_train_res_PhaseSolver)).real.detach().cpu().numpy(), ANN_ax_real[1], cmap='gray')
    plt.show()

    def PS(*args):
        global ANN_train_res_PhaseSolver
        return ANN_train_res_PhaseSolver

    n_sqr_Max = Dim_XY_P//2+1+Dim_XY_P%2
    n_sqr = n_sqr_Max
    Train_Sum_Epochs = 0
    Train_Sum_Time = 0

    one_T = torch.tensor([1]).to(torch.float32).to(device)
    test_res = PS(one_T, n_sqr_Max)

    print(test_res.size())
    ANN_fig_real, ANN_ax_real = plt.subplots(1,1,figsize=(28,6))
    plot_image(test_res.detach().cpu().numpy(), ANN_ax_real, cmap='twilight')
    plt.show()
    print()

    PCA_ANN = []
    PCA_ANN_LOSS = []
    PCA_ANN_HIO = []
    PCA_ANN_HIO_LOSS = []

    last_loss = float('inf')
    last_lr = 0.0001

    opt_fn = torch.optim.AdamW
    optimizer = opt_fn([ANN_train_res_PhaseSolver], lr=last_lr)

    one_T = torch.tensor([1]).to(torch.float32).to(device)
    def PS_Loss(f_phas_Ten_Sum, f_ampl_Ten = f_ampl_Ten, f_ampl_Ten_no_pad = f_ampl_Ten_no_pad, mask_Ten = mask_Ten, cheat_ampl_Ten = ampl_Ten):
        ANN_f_result_Ten = torch.polar(f_ampl_Ten, f_phas_Ten_Sum)

        ANN_comp_Ten = ifftn_shift(ANN_f_result_Ten)
        ANN_comp_Ten_no_pad = ANN_comp_Ten[Padding:-Padding, Padding:-Padding]
        loss_d_phase = MSE(torch.angle(ANN_comp_Ten_no_pad), torch.zeros_like(ANN_comp_Ten_no_pad.real))
        loss_nega = MSE(ANN_comp_Ten_no_pad.real[ANN_comp_Ten_no_pad.real<0], torch.zeros_like(ANN_comp_Ten_no_pad.real[ANN_comp_Ten_no_pad.real<0]))
        loss_nega = torch.nan_to_num(loss_nega)
        loss_imag = MSE(ANN_comp_Ten_no_pad.imag, torch.zeros_like(ANN_comp_Ten_no_pad.imag))
        loss_pad = MSE(ANN_comp_Ten.real[mask_Ten], torch.zeros_like(ANN_comp_Ten.real[mask_Ten])) *1000
        loss_cheat = MSE(ANN_comp_Ten.real*255,cheat_ampl_Ten*255)



        # Проверяем, что переворот изображения максимально отличается от самого изображения =========================================================== (*)
        # После ряда экспериментов, видим, что этот лосс дает около 8(+/-3) на инициализации
        loss_mirror = (1/MSE(ANN_comp_Ten_no_pad.real,torch.flip(ANN_comp_Ten_no_pad.real, [0, 1]))) * 0.1 # Корректирующий коэффицент

        loss = loss_pad + loss_nega + loss_imag + loss_d_phase
        return loss, loss_cheat, loss_pad, loss_d_phase, loss_mirror, loss_nega, loss_imag

    def train_model(epochs_Num):
      global Train_Sum_Epochs
      global Train_Sum_Time
      global optimizer
      global last_loss
      global last_lr
      global n_sqr
      global ANN_train_res_PhaseSolver

      ANN_f_phas_Arr_Last = PS(one_T, 1).detach().cpu().numpy()
      hio_flag = True

      for t in range(epochs_Num+1):  
          Epoch_Time = datetime.datetime.now()

          optimizer.zero_grad()

          optimizer.zero_grad()
          NN_out = PS(one_T, n_sqr)

          if MSE(ifftn_shift(torch.polar(f_ampl_Ten, NN_out)).real, ampl_Ten) > MSE(ifftn_shift(torch.polar(f_ampl_Ten, -NN_out)).real, ampl_Ten):
            NN_out = -NN_out

          loss, loss_cheat, loss_pad, loss_d_phas, loss_mirror, loss_nega, loss_imag = PS_Loss(NN_out)

          # PCA
          if n_sqr >= n_sqr_Max:
            PCA_ANN.append(NN_out.clone().detach().cpu().numpy())
            PCA_ANN_LOSS.append(loss_cheat.item())

          loss.backward()
          optimizer.step()

          delta_Time = datetime.datetime.now() - Epoch_Time
          Train_Sum_Time += (datetime.datetime.now() - Epoch_Time).total_seconds()
          Train_Sum_Epochs += 1

          if t % (epochs_Num/1000) == 0:
              if Save_ON:
                if t % (epochs_Num/50) == 0:
                    torch.save(PS, model_folder_Path+model_filename)
              print('%-4s' % (str(int(t/epochs_Num*1000))+'‰'),
                    '- Ep: '+ '%-5s'%str(Train_Sum_Epochs//10*10), 
                    '- LSum: '+'%10s'%('%.6f'%loss.item()), 
                    # '- LPha: '+'%10s'%('%.6f'%loss_phase.item()), 
                    # '- LIma: '+'%10s'%('%.6f'%loss_imag.item()), 
                    '- LD_CHEAT: '+'%10s'%('%.6f'%loss_cheat.item()), 
                    # '- LDPhas: '+'%10s'%('%.6f'%loss_d_phas.item()), 
                    '- LNega: '+'%10s'%('%.6f'%loss_nega.item()), 
                    '- LImag: '+'%10s'%('%.6f'%loss_imag.item()), 
                    '- LPad: '+'%10s'%('%.6f'%loss_pad.item()), 
                    # '- LMir: '+'%10s'%('%.6f'%loss_mirror.item()), 
                    '- Sec: '+'%4s'%'%.2f'%round(Train_Sum_Time,2), 
                    '- Log10(Lr): ' + '%2s' % str(int(-math.log10(last_lr))), 
                    '- Mem: ' + str(torch.cuda.memory_allocated('cuda')))


              if t % (epochs_Num/1000*10) == 0:
                  ANN_f_phas_Arr = NN_out.clone().detach().cpu().numpy()
                  ANN_real_Arr = ifftn_shift(torch.polar(f_ampl_Ten, NN_out)).real.detach().cpu().numpy()
                  ANN_f_phas_Arr_Delta = ANN_f_phas_Arr-ANN_f_phas_Arr_Last
                  ANN_f_phas_Arr_Delta_20 = ANN_f_phas_Arr_Delta.copy()
                  ANN_f_phas_Arr_Delta_20[abs(ANN_f_phas_Arr_Delta)<abs(ANN_f_phas_Arr_Delta).max()*0.8] = 0

                  # Выводим фазы фурье-образа картинок
                  ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,4, figsize=(28,6), subplot_kw={"projection": "3d"})
                  ANN_fig_f_phase.suptitle('Fourie Transform Phases; Квадрат: '+str(n_sqr*2)+'x'+str(n_sqr*2), fontsize=24)
                  plot_image_3D(f_phase_Ten.detach().cpu().numpy(), ANN_ax_f_phase[0], cmap='twilight', title="Оригинал")
                  plot_image_3D(ANN_f_phas_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Приближение(k)")
                  plot_image_3D(ANN_f_phas_Arr_Delta, ANN_ax_f_phase[2], cmap='twilight', title="Разница между k и k-10", alpha=0.5)
                  plot_image_3D(ANN_f_phas_Arr_Delta_20, ANN_ax_f_phase[3], cmap='twilight', title="Разница между k и k-10 (20% от max)", alpha=0.5)
                  plt.show()
                  print()
                  # Выводим фазы фурье-образа картинок
                  if n_sqr < Dim_XY_P//2+1:
                    print_ANN_f_phase_norm = torch.angle(torch.polar(f_ampl_Ten,NN_out))[n_sqr*2:-n_sqr*2,n_sqr*2:-n_sqr*2]
                    _pad = (Dim_XY_P-print_ANN_f_phase_norm.size(0))//2
                    print_ANN_f_phase_norm = torch.nn.functional.pad(print_ANN_f_phase_norm, (_pad,_pad,_pad,_pad)).detach().cpu().numpy()
                  else:
                    print_ANN_f_phase_norm = torch.angle(torch.polar(f_ampl_Ten,NN_out)).detach().cpu().numpy()
                  ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,5, figsize=(28,6))
                  ANN_fig_f_phase.suptitle('Fourie Transform Phases; Квадрат: '+str(n_sqr*2)+'x'+str(n_sqr*2), fontsize=24)
                  plot_image(ANN_real_Arr, ANN_ax_f_phase[0], cmap='gray', title="Приближение I")
                  plot_image(f_phase_Ten.detach().cpu().numpy(), ANN_ax_f_phase[1], cmap='twilight', title="Оригинал")
                  plot_image(print_ANN_f_phase_norm, ANN_ax_f_phase[2], cmap='twilight', title="Приближение I (-Pi, Pi)")
                  plot_image(ANN_f_phas_Arr, ANN_ax_f_phase[3], cmap='twilight', title="Приближение(k)")
                  plot_image(ANN_f_phas_Arr_Delta, ANN_ax_f_phase[4], cmap='twilight', title="Разница между k и k-10")
                  # plot_image(ANN_f_phas_Arr_Delta_20, ANN_ax_f_phase[4], cmap='twilight', title="Разница между k и k-10 (20% от max)")
                  plt.show()
                  print()

                  ANN_f_phas_Arr_Last = ANN_f_phas_Arr

              if loss.item() <= loss_break:
                  break

              if last_loss < loss.item():
                optimizer.zero_grad()
                last_lr = last_lr/10
                optimizer = opt_fn([ANN_train_res_PhaseSolver], lr=last_lr)
                optimizer.zero_grad()
                last_loss = float('inf')

              last_loss = loss.item()

    if Train_ON:
        Data_Str = '\nКартинки: ' + str(pics_Name) + '\nРазмер: ' + str(Dim_XY)+'x'+str(Dim_XY) + '\nПаддинг: ' + str(Padding)
        print(Data_Str)
        # !nvidia-smi
        train_model(epochs_Num//2)
        # train_model(10)

    phase_n = 150
    PCA_ANN_ = np.stack(PCA_ANN[::len(PCA_ANN)//phase_n][-phase_n:])
    PCA_ANN_LOSS_ = np.stack(PCA_ANN_LOSS[::len(PCA_ANN_LOSS)//phase_n][-phase_n:])
    print(PCA_ANN_.shape)

    pca_ann = PCA(n_components=2)
    PCA_ANN_ANS = pca_ann.fit_transform(PCA_ANN_.reshape(PCA_ANN_.shape[0], -1))
    print(PCA_ANN_ANS.shape)

    ANN_c = np.stack([[1.0,0,0]]*PCA_ANN_ANS.shape[0])
    ANN_c[:,0] = np.log(1 + ANN_c[:,0]*(1-(PCA_ANN_LOSS_-PCA_ANN_LOSS_.min())/(PCA_ANN_LOSS_.max()-PCA_ANN_LOSS_.min())))
    plt.scatter(PCA_ANN_ANS[:, 0], PCA_ANN_ANS[:, 1], c=ANN_c)
    plt.gca().set(
        title="ANN",
        xlabel="first feature",
        ylabel="second feature",
    )
    plt.show()


    PCA_ANN_HIO = []
    PCA_ANN_HIO_LOSS = []

    ANN_train_res_PhaseSolver = torch.from_numpy(ANN_train_res_PhaseSolver_init_Arr).clone().to(device)
    # ANN_train_res_PhaseSolver = HIO_func(ANN_train_res_PhaseSolver, hio_N=10000, era_N=10000)
    ANN_train_res_PhaseSolver.requires_grad_()



    import datetime
    last_loss = float('inf')
    last_lr = 0.0001

    optimizer = opt_fn([ANN_train_res_PhaseSolver], lr=last_lr)

    one_T = torch.tensor([1]).to(torch.float32).to(device)
    MSE = nn.MSELoss(reduction='mean')

    def train_model(epochs_Num):
      global Train_Sum_Epochs
      global Train_Sum_Time
      global optimizer
      global last_loss
      global last_lr
      global n_sqr
      global ANN_train_res_PhaseSolver

      ANN_f_phas_Arr_Last = PS(one_T, 1).detach().cpu().numpy()
      hio_flag = False

      for t in range(epochs_Num+1):  
          Epoch_Time = datetime.datetime.now()

          optimizer.zero_grad()
          if (t+1)%150 == 0:
            with torch.no_grad():
              ANN_train_res_PhaseSolver = HIO_func(ANN_train_res_PhaseSolver, hio_N=5, era_N=5)
            ANN_train_res_PhaseSolver.requires_grad_(True)

          optimizer.zero_grad()
          NN_out = PS(one_T, n_sqr)

          if MSE(ifftn_shift(torch.polar(f_ampl_Ten, NN_out)).real, ampl_Ten) > MSE(ifftn_shift(torch.polar(f_ampl_Ten, -NN_out)).real, ampl_Ten):
            NN_out = -NN_out

          # loss, loss_phase, loss_imag, loss_pad = PS_Loss(f_phas_Ten)
          # loss, loss_phase, loss_imag = PS_Loss(f_phas_Ten)
          # loss, loss_imag, loss_pad = PS_Loss(f_phas_Ten)
          loss, loss_cheat, loss_pad, loss_d_phas, loss_mirror, loss_nega, loss_imag = PS_Loss(NN_out)
          # _, loss_cheat, _, _, _ = PS_Loss(NN_out[10:-10,10:-10], )
          # loss = PS_Loss(f_phas_Ten)

          # PCA
          if n_sqr >= n_sqr_Max:
            PCA_ANN_HIO.append(NN_out.clone().detach().cpu().numpy())
            PCA_ANN_HIO_LOSS.append(loss_cheat.item())

          loss.backward()
          optimizer.step()

          delta_Time = datetime.datetime.now() - Epoch_Time
          Train_Sum_Time += (datetime.datetime.now() - Epoch_Time).total_seconds()
          Train_Sum_Epochs += 1  

          # if (abs(last_loss-loss.item()) < 0.001):
          #   if n_sqr < n_sqr_Max:
          #     n_sqr += 1
          #     last_lr = 0.0001
          #     optimizer.zero_grad()
          #     optimizer = opt_fn([ANN_train_res_PhaseSolver], lr=last_lr)
          #     optimizer.zero_grad()

          if t % (epochs_Num/1000) == 0:
              if Save_ON:
                if t % (epochs_Num/50) == 0:
                    torch.save(PS, model_folder_Path+model_filename)
              print('%-4s' % (str(int(t/epochs_Num*1000))+'‰'),
                    '- Ep: '+ '%-5s'%str(Train_Sum_Epochs//10*10), 
                    '- LSum: '+'%10s'%('%.6f'%loss.item()), 
                    # '- LPha: '+'%10s'%('%.6f'%loss_phase.item()), 
                    # '- LIma: '+'%10s'%('%.6f'%loss_imag.item()), 
                    '- LD_CHEAT: '+'%10s'%('%.6f'%loss_cheat.item()), 
                    # '- LDPhas: '+'%10s'%('%.6f'%loss_d_phas.item()), 
                    '- LNega: '+'%10s'%('%.6f'%loss_nega.item()), 
                    '- LImag: '+'%10s'%('%.6f'%loss_imag.item()), 
                    '- LPad: '+'%10s'%('%.6f'%loss_pad.item()), 
                    # '- LMir: '+'%10s'%('%.6f'%loss_mirror.item()), 
                    '- Sec: '+'%4s'%'%.2f'%round(Train_Sum_Time,2), 
                    '- Log10(Lr): ' + '%2s' % str(int(-math.log10(last_lr))), 
                    '- Mem: ' + str(torch.cuda.memory_allocated('cuda')))


              if t % (epochs_Num/1000*10) == 0:
                  ANN_f_phas_Arr = NN_out.clone().detach().cpu().numpy()
                  ANN_real_Arr = ifftn_shift(torch.polar(f_ampl_Ten, NN_out)).real.detach().cpu().numpy()
                  ANN_f_phas_Arr_Delta = ANN_f_phas_Arr-ANN_f_phas_Arr_Last
                  ANN_f_phas_Arr_Delta_20 = ANN_f_phas_Arr_Delta.copy()
                  ANN_f_phas_Arr_Delta_20[abs(ANN_f_phas_Arr_Delta)<abs(ANN_f_phas_Arr_Delta).max()*0.8] = 0

                  # Выводим фазы фурье-образа картинок
                  ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,4, figsize=(28,6), subplot_kw={"projection": "3d"})
                  ANN_fig_f_phase.suptitle('Fourie Transform Phases; Квадрат: '+str(n_sqr*2)+'x'+str(n_sqr*2), fontsize=24)
                  plot_image_3D(f_phase_Ten.detach().cpu().numpy(), ANN_ax_f_phase[0], cmap='twilight', title="Оригинал")
                  plot_image_3D(ANN_f_phas_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Приближение(k)")
                  plot_image_3D(ANN_f_phas_Arr_Delta, ANN_ax_f_phase[2], cmap='twilight', title="Разница между k и k-10", alpha=0.5)
                  plot_image_3D(ANN_f_phas_Arr_Delta_20, ANN_ax_f_phase[3], cmap='twilight', title="Разница между k и k-10 (20% от max)", alpha=0.5)
                  plt.show()
                  print()
                  # Выводим фазы фурье-образа картинок
                  if n_sqr < Dim_XY_P//2+1:
                    print_ANN_f_phase_norm = torch.angle(torch.polar(f_ampl_Ten,NN_out))[n_sqr*2:-n_sqr*2,n_sqr*2:-n_sqr*2]
                    _pad = (Dim_XY_P-print_ANN_f_phase_norm.size(0))//2
                    print_ANN_f_phase_norm = torch.nn.functional.pad(print_ANN_f_phase_norm, (_pad,_pad,_pad,_pad)).detach().cpu().numpy()
                  else:
                    print_ANN_f_phase_norm = torch.angle(torch.polar(f_ampl_Ten,NN_out)).detach().cpu().numpy()
                  ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,5, figsize=(28,6))
                  ANN_fig_f_phase.suptitle('Fourie Transform Phases; Квадрат: '+str(n_sqr*2)+'x'+str(n_sqr*2), fontsize=24)
                  plot_image(ANN_real_Arr, ANN_ax_f_phase[0], cmap='gray', title="Приближение I")
                  plot_image(f_phase_Ten.detach().cpu().numpy(), ANN_ax_f_phase[1], cmap='twilight', title="Оригинал")
                  plot_image(print_ANN_f_phase_norm, ANN_ax_f_phase[2], cmap='twilight', title="Приближение I (-Pi, Pi)")
                  plot_image(ANN_f_phas_Arr, ANN_ax_f_phase[3], cmap='twilight', title="Приближение(k)")
                  plot_image(ANN_f_phas_Arr_Delta, ANN_ax_f_phase[4], cmap='twilight', title="Разница между k и k-10")
                  # plot_image(ANN_f_phas_Arr_Delta_20, ANN_ax_f_phase[4], cmap='twilight', title="Разница между k и k-10 (20% от max)")
                  plt.show()
                  print()

                  ANN_f_phas_Arr_Last = ANN_f_phas_Arr

              if loss.item() <= loss_break:
                  break

              if last_loss < loss.item():
                if hio_flag:
                  optimizer.zero_grad()
                  with torch.no_grad():
                    ANN_train_res_PhaseSolver = HIO_func(ANN_train_res_PhaseSolver, hio_N=50, era_N=50)
                  ANN_train_res_PhaseSolver.requires_grad_(True)
                  optimizer.zero_grad()
                else:
                  optimizer.zero_grad()
                  last_lr = last_lr/10
                  optimizer = opt_fn([ANN_train_res_PhaseSolver], lr=last_lr)
                  optimizer.zero_grad()
                last_loss = float('inf')
                hio_flag = not hio_flag

              last_loss = loss.item()

    if Train_ON:
        Data_Str = '\nКартинки: ' + str(pics_Name) + '\nРазмер: ' + str(Dim_XY)+'x'+str(Dim_XY) + '\nПаддинг: ' + str(Padding)
        print(Data_Str)
        # !nvidia-smi
        train_model(epochs_Num//2)
        # train_model(10)

    phase_n = 150
    PCA_ANN_HIO_ = np.stack(PCA_ANN_HIO[::len(PCA_ANN_HIO)//phase_n][-phase_n:])
    PCA_ANN_HIO_LOSS_ = np.stack(PCA_ANN_HIO_LOSS[::len(PCA_ANN_HIO_LOSS)//phase_n][-phase_n:])
    print(PCA_ANN_HIO_.shape)

    pca_ann = PCA(n_components=2)
    PCA_ANN_HIO_ANS = pca_ann.fit_transform(PCA_ANN_HIO_.reshape(PCA_ANN_HIO_.shape[0], -1))
    print(PCA_ANN_HIO_ANS.shape)

    ANN_HIO_c = np.stack([[0.0,1.0,0]]*PCA_ANN_HIO_ANS.shape[0])
    ANN_HIO_c[:,1] = np.log(1 + ANN_HIO_c[:,1]*(1-(PCA_ANN_HIO_LOSS_-PCA_ANN_HIO_LOSS_.min())/(PCA_ANN_HIO_LOSS_.max()-PCA_ANN_HIO_LOSS_.min())))
    plt.scatter(PCA_ANN_HIO_ANS[:, 0], PCA_ANN_HIO_ANS[:, 1], c=ANN_HIO_c)
    plt.gca().set(
        title="ANN_HIO",
        xlabel="first feature",
        ylabel="second feature",
    )
    plt.show()


    if Save_ON:
        torch.save(PS, model_folder_Path+model_filename)

    def init_graphs(phase_shift):
        global ANN_real_Arr
        global ANN_imag_Arr
        global ANN_phase_Arr
        global ANN_f_ampl_Arr
        global ANN_f_phase_Arr
        global ANN_nopad_f_phase_Arr
        global nopad_real_Arr
        global nopad_imag_Arr
        global nopad_phase_Arr
        global nopad_f_phase_Arr
        global f_phase_Arr

        # Сдвиг фазы для верного сопостовления действительной и мнимой частей
        #@markdown Сдвиг фазы происходит на pi/2 * phase_shift
        # Подготавливаем данные
        ANN_phase_res = PS(one_T, Dim_XY_P//2+1)+math.pi/2*phase_shift
        ANN_f_phase_Arr = ANN_phase_res.detach().cpu().numpy()
        ANN_f_result_Ten = torch.polar(f_ampl_Ten,ANN_phase_res)
        ANN_result_Ten = ifftn_shift(ANN_f_result_Ten)
        ANN_f_ampl_Arr = fftn_shift(ANN_result_Ten).abs().detach().cpu().numpy()
        if Padding > 0: # Убираем паддинг
            ANN_result_Ten = ANN_result_Ten[Padding:-Padding, Padding:-Padding]

        ANN_real_Ten = ANN_result_Ten.real.cpu().detach()
        ANN_imag_Ten = ANN_result_Ten.imag.cpu().detach()
        ANN_phase_Ten = torch.angle(ANN_result_Ten).cpu().detach()
        # ANN_real_Ten = torch.nn.functional.interpolate(ANN_real_Ten.unsqueeze(0).unsqueeze(0), scale_factor=(Dim_XY/ANN_real_Ten.size(0),Dim_XY/ANN_real_Ten.size(1)), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
        # ANN_imag_Ten = torch.nn.functional.interpolate(ANN_imag_Ten.unsqueeze(0).unsqueeze(0), scale_factor=(Dim_XY/ANN_imag_Ten.size(0),Dim_XY/ANN_imag_Ten.size(1)), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)


        ANN_real_Arr = ANN_real_Ten.numpy()
        ANN_imag_Arr = ANN_imag_Ten.numpy()
        ANN_phase_Arr = ANN_phase_Ten.numpy()

        ANN_f_phase_Arr = torch.angle(ANN_f_result_Ten).cpu().detach().numpy()

        if Padding > 0:
            nopad_phase_Arr = phase_Ten_no_pad.cpu().detach().numpy()
            nopad_real_Arr = real_Arr[Padding:-Padding, Padding:-Padding]
            nopad_imag_Arr = imag_Arr[Padding:-Padding, Padding:-Padding]
        else: 
            nopad_phase_Arr = phase_Ten_no_pad.cpu().detach().numpy()
            nopad_real_Arr = real_Arr
            nopad_imag_Arr = imag_Arr

        ANN_f_all_Ten = fftn_shift(torch.from_numpy(ANN_real_Arr + ANN_imag_Arr*1j))
        ANN_nopad_f_phase_Arr = torch.angle(ANN_f_all_Ten).numpy()

        f_phase_Arr = f_phase_Ten.cpu().detach().numpy()
        nopad_f_phase_Arr = f_phase_Ten_no_pad.cpu().detach().numpy()

    #@title Функция вывода
    #@markdown
    Delta_Range = ()
    Delta_Range = list(Delta_Range)

    norm_const = 1

    def graph_print():
        right_phase_shift = 0
        min_mean = float('inf')

        # for phase_shift in range(4):
        #     init_graphs(phase_shift)
        #     mean = np.mean(ring_alg(ANN_f_phase_Arr, nopad_f_phase_Arr)).item(0)/math.pi*100
        #     if min_mean > mean:
        #         right_phase_shift = phase_shift
        #         min_mean = mean
        # init_graphs(right_phase_shift)
        init_graphs(0)

        # Выводим параметры
        # Data_Str = 'Картинки: ' + str(pics_Name) + '\nРазмер: ' + str(Dim_XY) + '\nПаддинг: ' + str(Padding)
        Data_Str = str(pics_Name) + ';  Размер: ' + str(Dim_XY) + ';  Паддинг: ' + str(Padding) + ';  Эпохи: ' + str(Train_Sum_Epochs//10*10) + ';  Время: ' + str(round(Train_Sum_Time))+' сек'
        Data_fig, Data_ax = plt.subplots(figsize=(26,1))
        Data_ax.get_xaxis().set_visible(False)
        Data_ax.get_yaxis().set_visible(False)
        Data_fig.text(0.2, 0.5, Data_Str, ha='left', va='center', size=35)
        plt.show()
        print() 


        # Выводим действ. картинки
        ANN_fig_real, ANN_ax_real = plt.subplots(1,3, figsize=(28,6))
        ANN_fig_real.suptitle('Real part', fontsize=24)

        plot_image(np.abs(ANN_real_Arr), ANN_ax_real[0], cmap='gray', title="Result")
        plot_image(np.flip(np.abs(ANN_real_Arr), (0, 1)), ANN_ax_real[1], cmap='gray', title="Result 180 гр")
        plot_image(nopad_real_Arr, ANN_ax_real[2], cmap='gray', title="Original")
        plt.show()
        print() 

        # Выводим фазы фурье-образа картинок
        ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,3, figsize=(28,6))
        ANN_fig_f_phase.suptitle('Fourie Transform Phases', fontsize=24)
        plot_image(ANN_f_phase_Arr, ANN_ax_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
        plot_image(f_phase_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
        plot_image(ring_alg(ANN_f_phase_Arr, f_phase_Arr), ANN_ax_f_phase[2], cmap='gray', title="Delta", vmin=0, vmax=math.pi)
        plt.show()
        print()

        # Выводим фазы фурье-образа картинок без паддинга
        ANN_fig_f_nopad_phase, ANN_ax_nopad_f_phase = plt.subplots(1,3, figsize=(28,6))
        ANN_fig_f_nopad_phase.suptitle('Fourie Transform Phases No Padding', fontsize=24)
        plot_image(ANN_nopad_f_phase_Arr, ANN_ax_nopad_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
        plot_image(nopad_f_phase_Arr, ANN_ax_nopad_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
        plot_image(ring_alg(ANN_nopad_f_phase_Arr, nopad_f_phase_Arr), ANN_ax_nopad_f_phase[2], cmap='gray', title="Delta", vmin=0, vmax=math.pi)
        plt.show()
        print() 

        Delta_Range.append(round(np.mean(ring_alg(ANN_f_phase_Arr, f_phase_Arr)).item(0)/math.pi*100))

    graph_print()

    print(Delta_Range)

    def random_start(amplitudes): # Генерируем T1* по случайным фазам 
        random_phase = math.pi * (torch.rand(amplitudes.size()) * 2 - 1)
        return torch.polar(amplitudes.to(device), random_phase.to(device))


    def iter_mask(real_data, real_mask): # Создаем маску иттерации - помечаем всё прямое пространство < 0 и паддинги
        return torch.logical_or(real_data<0, real_mask) 


    def real_support_er(real_data, real_mask): # Зануляем тензор по маске
        new_real_data = real_data.clone()
        new_real_data[real_mask] = 0
        return new_real_data


    def real_support_hio(real_data, real_mask, prev_real_data, beta):
        new_real_data = real_data.clone()
        new_real_data[real_mask] = prev_real_data[real_mask] - (1-beta) * new_real_data[real_mask]
        return new_real_data


    def real_error(real_data, real_mask):
        return torch.sum(torch.abs(real_data[real_mask]) ** 2)/torch.sum(torch.abs(real_data[real_mask == 0]) ** 2)


    def recip_support(recip_data, amplitudes): # Применяем угаданные на итерации фазы к реальным амплитудам
        phases = torch.angle(recip_data)
        return torch.polar(amplitudes, phases)

    def recip_error(recip_data, amplitudes):
        return torch.sqrt(torch.sum(torch.abs(recip_data) - amplitudes) ** 2 / torch.sum(amplitudes ** 2))


    T1_c_pri = random_start(f_ampl_Ten)
    T1_c_pri = torch.from_numpy(ANN_train_res_PhaseSolver_init_Arr)
    T1_pri = torch.abs(T1_c_pri)
    TLenna_c_pri = ifftn_shift(T1_c_pri)
    TLenna_pri = torch.abs(TLenna_c_pri)
    real_result_sup = real_support_er(TLenna_pri, mask_Ten)

    fig, ax = plt.subplots(figsize=(6,6))
    plot_image(real_result_sup.real.cpu().detach().numpy(), ax, logscale=False, cmap='gray')

    from re import X
    # T1_c_pri = random_start(f_ampl_Ten).to(device)
    T1_c_pri = torch.polar(f_ampl_Ten,torch.from_numpy(ANN_train_res_PhaseSolver_init_Arr).to(device))

    n_hio = 400
    n_er = 0
    n_repeat = 1

    n_total = n_repeat*(n_hio + n_er)

    beta_start = 0.1
    beta_end = 0.1

    real_error_vals = np.zeros(n_total)
    recip_error_vals = np.zeros(n_total)

    report_step = n_total//6
    real_report = []
    recip_report = []
    PCA_HIO = []
    PCA_HIO_LOSS = []

    prev_TLenna_c_pri = ifftn_shift(T1_c_pri)
    x = prev_TLenna_c_pri
    x_prev = x

    PCA_HIO = []
    PCA_HIO_LOSS = []

    PCA_HIO.append(fftn_shift(x).angle().detach().cpu().numpy())
    loss, loss_cheat, loss_pad, loss_d_phas, loss_mirror, loss_nega, loss_imag = PS_Loss(fftn_shift(x).angle())
    PCA_HIO_LOSS.append(loss_cheat.detach().item())

    Train_Sum_Time_HIO = 0

    for i in range(n_total):

        Epoch_Time = datetime.datetime.now()
        hio = (i % (n_hio + n_er)) < n_hio
        beta = beta_start + (beta_end - beta_start)*i/n_total

        new_T1_c_pri = fftn_shift(x)
        recip_error_vals[i] = recip_error(new_T1_c_pri, f_ampl_Ten).detach().item()

        new_T1_c_pri_phase = new_T1_c_pri.angle()
        if MSE(ifftn_shift(torch.polar(f_ampl_Ten, new_T1_c_pri_phase)).real, ampl_Ten) > MSE(ifftn_shift(torch.polar(f_ampl_Ten, -new_T1_c_pri_phase)).real, ampl_Ten):
            new_T1_c_pri_phase = -new_T1_c_pri_phase

        PCA_HIO.append(new_T1_c_pri.angle().detach().cpu().numpy())
        loss, loss_cheat, loss_pad, loss_d_phas, loss_mirror, loss_nega, loss_imag = PS_Loss(new_T1_c_pri.angle())
        PCA_HIO_LOSS.append(loss_cheat.detach().item())

        new_T1_c_res = torch.polar(f_ampl_Ten, new_T1_c_pri_phase)
        TLenna_c_pri = ifftn_shift(new_T1_c_res)
        TLenna_pri = TLenna_c_pri.real
        it_mask = iter_mask(TLenna_pri, mask_Ten)


    #     PCA_HIO.append(fftn_shift(TLenna_pri).angle().detach().cpu().numpy())
    #     loss, loss_cheat, loss_pad, loss_d_phas, loss_mirror, loss_nega, loss_imag = PS_Loss(fftn_shift(TLenna_pri).angle())
    #     PCA_HIO_LOSS.append(loss_cheat.detach().item())

        if hio:
            TLenna_c_res = real_support_hio(TLenna_c_pri, it_mask, x, beta)
        else:
            TLenna_c_res = real_support_er(TLenna_c_pri, it_mask)

        real_error_vals[i] = real_error(TLenna_c_res, it_mask).detach().item()

        x = TLenna_c_res
        delta_Time = datetime.datetime.now() - Epoch_Time
        Train_Sum_Time_HIO += (datetime.datetime.now() - Epoch_Time).total_seconds()

        print('%-4s' % (str(int(i/epochs_Num*1000))+'‰'),
                    '- Ep: '+ '%-5s'%str(i//10*10), 
                    '- LSum: '+'%10s'%('%.6f'%loss.item()), 
                    # '- LPha: '+'%10s'%('%.6f'%loss_phase.item()), 
                    # '- LIma: '+'%10s'%('%.6f'%loss_imag.item()), 
                    '- LD_CHEAT: '+'%10s'%('%.6f'%loss_cheat.item()), 
                    # '- LDPhas: '+'%10s'%('%.6f'%loss_d_phas.item()), 
                    '- LNega: '+'%10s'%('%.6f'%loss_nega.item()), 
                    '- LImag: '+'%10s'%('%.6f'%loss_imag.item()), 
                    '- LPad: '+'%10s'%('%.6f'%loss_pad.item()),
                    '- Sec: '+'%4s'%'%.2f'%round(Train_Sum_Time_HIO,2)
              )

        if (i % 10 == 0):
          ANN_f_phas_Arr = fftn_shift(x.real).angle().detach().cpu().numpy()
          ANN_real_Arr = x.real.detach().cpu().numpy()
          ANN_f_phas_Arr_Delta = ANN_f_phas_Arr-fftn_shift(x_prev.real).angle().detach().cpu().numpy()
          ANN_f_phas_Arr_Delta_20 = ANN_f_phas_Arr_Delta.copy()
          ANN_f_phas_Arr_Delta_20[abs(ANN_f_phas_Arr_Delta)<abs(ANN_f_phas_Arr_Delta).max()*0.8] = 0

          # Выводим фазы фурье-образа картинок
          ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,4, figsize=(28,6), subplot_kw={"projection": "3d"})
          ANN_fig_f_phase.suptitle('Fourie Transform Phases (HIO+ERA)', fontsize=24)
          plot_image_3D(f_phase_Ten.detach().cpu().numpy(), ANN_ax_f_phase[0], cmap='twilight', title="Оригинал")
          plot_image_3D(ANN_f_phas_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Приближение(k)")
          plot_image_3D(ANN_f_phas_Arr_Delta, ANN_ax_f_phase[2], cmap='twilight', title="Разница между k и k-10", alpha=0.5)
          plot_image_3D(ANN_f_phas_Arr_Delta_20, ANN_ax_f_phase[3], cmap='twilight', title="Разница между k и k-10 (20% от max)", alpha=0.5)
          plt.show()
          print()
          # Выводим фазы фурье-образа картинок
          ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,4, figsize=(28,6))
          ANN_fig_f_phase.suptitle('Fourie Transform Phases (HIO+ERA)', fontsize=24)
          plot_image(ANN_real_Arr, ANN_ax_f_phase[0], cmap='gray', title="Приближение(k) I")
          plot_image(ANN_f_phas_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Приближение(k)")
          plot_image(ANN_f_phas_Arr_Delta, ANN_ax_f_phase[2], cmap='twilight', title="Разница между k и k-10")
          plot_image(ANN_f_phas_Arr_Delta_20, ANN_ax_f_phase[3], cmap='twilight', title="Разница между k и k-10 (20% от max)")
          plt.show()
          print()

          x_prev = x
        # print('{} r:{:2.4f} f:{:2.4f}'.format(i, real_error_vals[i], recip_error_vals[i]))
        if ((i+1)%report_step == 0):
            real_report.append(TLenna_c_pri.cpu().detach().numpy())
            recip_report.append(T1_c_pri.cpu().detach().numpy())


    fig, ax = plt.subplots(2, 3, figsize=(14,8))
    for i, axis in enumerate(ax.flatten()):
        plot_image(np.abs(real_report[i]), axis, logscale=False, cmap='gray')
        # axis.set_xlim([5, Dim_XY-5])
        # axis.set_ylim([100, 356])

    loss, loss_cheat, loss_pad, loss_d_phas, loss_mirror, loss_nega, loss_imag = PS_Loss(fftn_shift(torch.from_numpy(real_report[-1]).abs().to(device)).angle())
    print(
                    '- LSum: '+'%10s'%('%.6f'%loss.item()), 
                    '- LDPhas: '+'%10s'%('%.6f'%loss_d_phas.item()), 
                    '- LPad: '+'%10s'%('%.6f'%loss_pad.item()), 
    )

    HIO_real_Ten = ifftn_shift(torch.polar(f_ampl_Ten, fftn_shift(torch.from_numpy(real_report[-1]).to(device)).angle())).real
    HIO_real_Ten = torch.flip(HIO_real_Ten, [0,1])

    HIO_real_Arr = HIO_real_Ten[Padding:-Padding,Padding:-Padding].real.detach().cpu().numpy()
    HIO_f_phase_Arr = fftn_shift(HIO_real_Ten).angle().detach().cpu().numpy()
    if Padding > 0:
      HIO_nopad_f_phase_Arr = fftn_shift(HIO_real_Ten[Padding:-Padding,Padding:-Padding]).angle().detach().cpu().numpy()
    else:
      HIO_nopad_f_phase_Arr = HIO_f_phase_Arr


    # Выводим действ. картинки
    ANN_fig_real, ANN_ax_real = plt.subplots(1,3, figsize=(28,6))
    ANN_fig_real.suptitle('Real part', fontsize=24)

    plot_image(np.abs(HIO_real_Arr), ANN_ax_real[0], cmap='gray', title="Result")
    plot_image(np.flip(np.abs(HIO_real_Arr), (0, 1)), ANN_ax_real[1], cmap='gray', title="Result 180 гр")
    plot_image(nopad_real_Arr, ANN_ax_real[2], cmap='gray', title="Original")
    plt.show()
    print() 

    # Выводим фазы фурье-образа картинок
    ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,3, figsize=(28,6), subplot_kw={"projection": "3d"})
    # ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,3, figsize=(28,6))
    ANN_fig_f_phase.suptitle('Fourie Transform Phases', fontsize=24)
    plot_image_3D(HIO_f_phase_Arr, ANN_ax_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
    plot_image_3D(f_phase_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
    plot_image_3D(ring_alg(HIO_f_phase_Arr, f_phase_Arr), ANN_ax_f_phase[2], cmap='gray', title="Delta", vmin=0, vmax=math.pi)
    # plot_image(HIO_f_phase_Arr, ANN_ax_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
    # plot_image(f_phase_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
    # plot_image(ring_alg(HIO_f_phase_Arr, f_phase_Arr), ANN_ax_f_phase[2], cmap='gray', title="Delta", vmin=0, vmax=math.pi)
    plt.show()
    print()

    # Выводим фазы фурье-образа картинок без паддинга
    ANN_fig_f_nopad_phase, ANN_ax_nopad_f_phase = plt.subplots(1,3, figsize=(28,6), subplot_kw={"projection": "3d"})
    # ANN_fig_f_nopad_phase, ANN_ax_nopad_f_phase = plt.subplots(1,3, figsize=(28,6))
    ANN_fig_f_nopad_phase.suptitle('Fourie Transform Phases No Padding', fontsize=24)
    plot_image_3D(HIO_nopad_f_phase_Arr, ANN_ax_nopad_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
    plot_image_3D(nopad_f_phase_Arr, ANN_ax_nopad_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
    plot_image_3D(ring_alg(HIO_nopad_f_phase_Arr, nopad_f_phase_Arr), ANN_ax_nopad_f_phase[2], cmap='gray', title="Delta", vmin=0, vmax=math.pi)
    # plot_image(HIO_nopad_f_phase_Arr, ANN_ax_nopad_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
    # plot_image(nopad_f_phase_Arr, ANN_ax_nopad_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
    # plot_image(ring_alg(HIO_nopad_f_phase_Arr, nopad_f_phase_Arr), ANN_ax_nopad_f_phase[2], cmap='gray', title="Delta", vmin=0, vmax=math.pi)
    plt.show()
    print() 


    phase_n = 150
    PCA_HIO_ = np.stack(PCA_HIO[::len(PCA_HIO)//phase_n][-phase_n:])
    PCA_HIO_LOSS_ = np.stack(PCA_HIO_LOSS[::len(PCA_HIO_LOSS)//phase_n][-phase_n:])
    print(PCA_HIO_.shape)

    print(MSE(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), torch.from_numpy(PCA_HIO_[-1]))).real*255, ampl_Ten.detach().cpu()*255), PCA_HIO_LOSS_[-1])

    pca_hio = PCA(n_components=2)
    PCA_HIO_ANS = pca_hio.fit_transform(PCA_HIO_.reshape(PCA_HIO_.shape[0], -1))
    # PCA_HIO_ANS = np.array(PCA_HIO_ANS)
    print(PCA_HIO_ANS.shape)

    HIO_c = np.stack([[0.0,0.0,1.0]]*PCA_HIO_ANS.shape[0])
    HIO_c[:,2] = np.log(1 + HIO_c[:,2]*(1-(PCA_HIO_LOSS_-PCA_HIO_LOSS_.min())/(PCA_HIO_LOSS_.max()-PCA_HIO_LOSS_.min())))
    plt.scatter(PCA_HIO_ANS[:, 0], PCA_HIO_ANS[:, 1], c=HIO_c)
    plt.gca().set(
        aspect="equal",
        title="HIO",
        xlabel="first feature",
        ylabel="second feature",
    )
    plt.show()


    PCA_ALL_ = np.concatenate((PCA_ANN_, PCA_ANN_HIO_, PCA_HIO_, ANN_train_res_PhaseSolver_init_Arr.reshape(1,Dim_XY_P,Dim_XY_P), f_phase_Ten.detach().cpu().numpy().reshape(1,Dim_XY_P,Dim_XY_P)))
    print(PCA_ALL_.shape)
    print(PCA_ALL_[-2]) 
    print(PCA_ALL_[-1]) 
    # pca_all = PCA(n_components=2)
    def angle_metric(X1,X2):
      X = np.square(
              np.abs(
                  np.subtract(
                    np.exp(X1*1j), np.exp(X2*1j)
                  )
              )
          ).mean()
      return X

    pca_dim_n = 2

    pca_all = TSNE(n_components=pca_dim_n, metric=angle_metric)
    PCA_ALL_ANS = pca_all.fit_transform(PCA_ALL_.reshape(PCA_ALL_.shape[0], -1))

    torch.save(torch.from_numpy(PCA_ALL_ANS), 'ALL RAND_INIT ' +str(iiii)+ '.pt')

    print(PCA_ALL_ANS.shape)

    PCA_ALL_ANS_ANN = PCA_ALL_ANS[0:phase_n]
    PCA_ALL_ANS_ANN_HIO = PCA_ALL_ANS[phase_n:phase_n*2]
    PCA_ALL_ANS_HIO = PCA_ALL_ANS[phase_n*2:-2]

    PCA_ALL_ANS_ANN = PCA_ALL_ANS[0:phase_n]
    PCA_ALL_ANS_ANN_HIO = PCA_ALL_ANS[phase_n:phase_n*2]
    PCA_ALL_ANS_HIO = PCA_ALL_ANS[phase_n*2:-2]

    f_grid = plt.figure(1, figsize=(14,20))
    if pca_dim_n == 3:
      ax_grid = f_grid.add_subplot(projection='3d')
    elif pca_dim_n == 2:
      ax_grid = f_grid.add_subplot()


    ANN_fig_real.suptitle("ORIG", fontsize=20)

    # ax_grid.scatter(PCA_ALL_GRID_X, PCA_ALL_GRID_Y, c=ALL_GRID_c)
    ax_grid.scatter(PCA_ALL_ANS_HIO[:,0], PCA_ALL_ANS_HIO[:,1], c=HIO_c, label="HIO")
    ax_grid.scatter(PCA_ALL_ANS_ANN[:,0], PCA_ALL_ANS_ANN[:,1], c=ANN_c, label="SGD")
    ax_grid.scatter(PCA_ALL_ANS_ANN_HIO[:,0], PCA_ALL_ANS_ANN_HIO[:,1], c=ANN_HIO_c, label="SGD+HIO")
    ax_grid.scatter(PCA_ALL_ANS_HIO[0,0], PCA_ALL_ANS_HIO[0,1], c='cyan', label="HIO", marker='^')
    ax_grid.scatter(PCA_ALL_ANS_ANN[0,0], PCA_ALL_ANS_ANN[0,1], c="orange", label="SGD", marker='^')
    ax_grid.scatter(PCA_ALL_ANS_ANN_HIO[0,0], PCA_ALL_ANS_ANN_HIO[0,1], c="lime", label="SGD+HIO", marker='^')
    ax_grid.scatter(PCA_ALL_ANS[-2,0], PCA_ALL_ANS[-2,1], c='gray', label="Init", marker='^')
    ax_grid.scatter(PCA_ALL_ANS[-1,0], PCA_ALL_ANS[-1,1], c='purple', label="True", marker='^')


    if pca_dim_n == 3:
      ax_grid.set(
        title="HIO+SGD",
        xlabel="first feature",
        ylabel="second feature",
        zlabel="third feature",
      ) 
    elif pca_dim_n == 2:
      ax_grid.set(
        title="HIO+SGD",
        xlabel="first feature",
        ylabel="second feature",
      ) 

    plt.show()

    print('ORIG SGD:', math.sqrt(
        np.square(
            np.abs(
                np.subtract(
                    np.exp(PCA_ALL_[phase_n]*1j), np.exp(PCA_ALL_[-1]*1j)
                )
            )
        ).mean())
    )
    print('ORIG HIO:', math.sqrt(
        np.square(
            np.abs(
                np.subtract(
                    np.exp(PCA_ALL_[-phase_n]*1j), np.exp(PCA_ALL_[-1]*1j)
                )
            )
        ).mean())
    )

    print('PCA SGD<-->GTRUTH DISTANCE:', math.sqrt(np.square(PCA_ALL_ANS_ANN[-1] - PCA_ALL_ANS[-1]).sum()))
    print('PCA HIO<-->GTRUTH DISTANCE:', math.sqrt(np.square(PCA_ALL_ANS_HIO[-1] - PCA_ALL_ANS[-1]).sum()))


    PCA_HIO_DIST_TO_TRUTH = np.sqrt(
      np.square(
        np.sum(
          np.subtract(
              PCA_ALL_ANS_HIO, PCA_ALL_ANS[-1]
          ),
          axis=-1
        )
      )
    )
    PCA_ANN_HIO_DIST_TO_TRUTH = np.sqrt(
      np.square(
        np.sum(
          np.subtract(
              PCA_ALL_ANS_ANN_HIO, PCA_ALL_ANS[-1]
          ),
          axis=-1
        )
      )
    )
    PCA_ANN_DIST_TO_TRUTH = np.sqrt(
      np.square(
        np.sum(
          np.subtract(
              PCA_ALL_ANS_ANN, PCA_ALL_ANS[-1]
          ),
          axis=-1
        )
      )
    )

    fig, ax = plt.subplots(figsize=(20,7))
    ax.set(
        title="HIO+SGD",
        xlabel="Итерация",
        ylabel="Расстояние до истинной фазы"
      ) 
    f_nnn_ = 0
    nnn_ = -1
    plt.plot(PCA_HIO_DIST_TO_TRUTH[f_nnn_:nnn_], color='blue')
    plt.plot(PCA_ANN_HIO_DIST_TO_TRUTH[f_nnn_:nnn_], color='green')
    plt.plot(PCA_ANN_DIST_TO_TRUTH[f_nnn_:nnn_], color='red')
    plt.show()
    # plt.plot(PCA_ANN_LOSS_, color='orange')
    plt.plot(PCA_ANN_HIO_LOSS_[f_nnn_:nnn_], color='lime')
    plt.plot(PCA_HIO_LOSS_[f_nnn_:nnn_], color='cyan')
    plt.show()
    
    ann_final, ann_hio_final, hio_final = torch.from_numpy(PCA_ANN_[-1]), torch.from_numpy(PCA_ANN_HIO_[-1]) ,torch.from_numpy(PCA_HIO_[-1])
    hio_final = fftn_shift(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), hio_final.detach().cpu())).real).angle()
    ann_hio_final = fftn_shift(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), ann_hio_final)).real).angle()
    ann_final = fftn_shift(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), ann_final.detach().cpu())).real).angle()

    print('HIO:', MSE(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), hio_final.detach().cpu())).real*255, ampl_Ten.detach().cpu()*255).item(), PCA_HIO_LOSS_[-1])
    print('HIO+SGD:', MSE(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), ann_hio_final.detach().cpu())).real*255, ampl_Ten.detach().cpu()*255).item(), PCA_ANN_HIO_LOSS_[-1])
    print('SGD:', MSE(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), ann_final.detach().cpu())).real*255, ampl_Ten.detach().cpu()*255).item(), PCA_ANN_LOSS_[-1])

    ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(2,4, figsize=(18,6))
    plot_image(ann_final.detach().cpu().numpy().real, ANN_ax_f_phase[0][0], cmap='twilight', title = "SGD")
    plot_image(ann_hio_final.detach().cpu().numpy().real, ANN_ax_f_phase[0][1], cmap='twilight', title = "SGD+HIO")
    plot_image(hio_final.detach().cpu().numpy().real, ANN_ax_f_phase[0][2], cmap='twilight', title = "HIO")
    plot_image(f_phase_Ten.detach().cpu().numpy().real, ANN_ax_f_phase[0][3], cmap='twilight', title = "ORIG")
    plot_image(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), ann_final)).detach().cpu().numpy().real, ANN_ax_f_phase[1][0], cmap='gray')
    plot_image(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), ann_hio_final)).detach().cpu().numpy().real, ANN_ax_f_phase[1][1], cmap='gray')
    plot_image(ifftn_shift(torch.polar(f_ampl_Ten.detach().cpu(), hio_final)).detach().cpu().numpy().real, ANN_ax_f_phase[1][2], cmap='gray')
    plot_image(ampl_Ten.detach().cpu().numpy().real, ANN_ax_f_phase[1][3], cmap='gray')

    ANN_fig_f_phase.show()